# Text Main

> This module contains the main Python class for data control: `TextDataController`

- skip_showdoc: true
- skip_exec: true

In [ ]:
#| default_exp text_main

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from __future__ import annotations
from sklearn.preprocessing import LabelEncoder,MultiLabelBinarizer
from datasets import DatasetDict,Dataset,IterableDataset,load_dataset,concatenate_datasets,Value
from pathlib import Path
from that_nlp_library.utils import *
from functools import partial
import warnings

In [ ]:
import pandas as pd
import numpy as np
from that_nlp_library.text_transformation import *
from that_nlp_library.text_augmentation import *
from importlib.machinery import SourceFileLoader
import os

## Content Transformation, Augmentations, and Tokenization

In [ ]:
#| export
def tokenizer_explain(inp, # Input sentence
                      tokenizer, # Tokenizer (preferably from HuggingFace)
                      split_word=False # Is input `inp` split into list or not
                     ):
    "Display results from tokenizer"
    print('\t\t------- Tokenizer Explained -------')
    print('----- Input -----')
    print(inp)
    print()
    print('----- Tokenized results ----- ')
    print(tokenizer(inp,is_split_into_words=split_word))
    print()
    tok = tokenizer.encode(inp,is_split_into_words=split_word)
    print('----- Results from tokenizer.convert_ids_to_tokens -----')
    print(tokenizer.convert_ids_to_tokens(tok))
    print()
    print('----- Results from tokenizer.decode ----- ')
    print(tokenizer.decode(tok))
    print()

In [ ]:
show_doc(tokenizer_explain)

---

[source](https://github.com/anhquan0412/that-nlp-library/blob/main/that_nlp_library/text_main.py#L19){target="_blank" style="float:right; font-size:smaller"}

### tokenizer_explain

>      tokenizer_explain (inp, tokenizer, split_word=False)

Display results from tokenizer

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| inp |  |  | Input sentence |
| tokenizer |  |  | Tokenizer (preferably from HuggingFace) |
| split_word | bool | False | Is input `inp` split into list or not |

Let's load a tokenizer from EnviBert model. Uncomment the command line below to download necessary files to build this tokenizer

In [ ]:
# !pip install gdown

In [ ]:
# !gdown 14X9fGijA7kdNfe4dM_8gqfxIWtj1Q-hb -O ./envibert_cache --folder

In [ ]:
cache_dir=Path('./envibert_tokenizer')
_tokenizer = SourceFileLoader("envibert.tokenizer", 
                             str(cache_dir/'envibert_tokenizer.py')).load_module().RobertaTokenizer(cache_dir)

Note that Envibert tokenizer does not required the input to be tokenized using word_tokenize from UnderTheSea library

In [ ]:
inp = 'Hội cư dân   chung cư sen hồng- chung cư    lotus sóng thần thủ đức. Thủ Đức là một huyện trực thuộc thành phố Hồ Chí Minh'
tokenizer_explain(inp,_tokenizer,split_word=False)

		------- Tokenizer Explained -------
----- Input -----
Hội cư dân   chung cư sen hồng- chung cư    lotus sóng thần thủ đức. Thủ Đức là một huyện trực thuộc thành phố Hồ Chí Minh

----- Tokenized results ----- 
{'input_ids': [0, 857, 1033, 191, 664, 1033, 7366, 2615, 142, 664, 1033, 671, 1355, 2294, 993, 413, 2900, 244, 1019, 827, 24, 40, 647, 773, 549, 119, 511, 1134, 1690, 758, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

----- Results from tokenizer.convert_ids_to_tokens -----
['<s>', '▁Hội', '▁cư', '▁dân', '▁chung', '▁cư', '▁sen', '▁hồng', '-', '▁chung', '▁cư', '▁lot', 'us', '▁sóng', '▁thần', '▁thủ', '▁đức', '.', '▁Thủ', '▁Đức', '▁là', '▁một', '▁huyện', '▁trực', '▁thuộc', '▁thành', '▁phố', '▁Hồ', '▁Chí', '▁Minh', '</s>']

----- Results from tokenizer.decode ----- 
<s> ▁Hội ▁cư ▁dân ▁chung ▁cư ▁sen ▁

In [ ]:
inp = ['hội', 'cư', 'dân', 'chung', 'cư', 'sen', 'hồng', '-', 'chung', 'cư', 'lotus', 'sóng', 'thần', 'thủ', 'đức']
tokenizer_explain(inp,_tokenizer,split_word=True)

		------- Tokenizer Explained -------
----- Input -----
['hội', 'cư', 'dân', 'chung', 'cư', 'sen', 'hồng', '-', 'chung', 'cư', 'lotus', 'sóng', 'thần', 'thủ', 'đức']

----- Tokenized results ----- 
{'input_ids': [0, 227, 1033, 191, 664, 1033, 7366, 2615, 13, 664, 1033, 671, 1355, 2294, 993, 413, 2900, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

----- Results from tokenizer.convert_ids_to_tokens -----
['<s>', '▁hội', '▁cư', '▁dân', '▁chung', '▁cư', '▁sen', '▁hồng', '▁-', '▁chung', '▁cư', '▁lot', 'us', '▁sóng', '▁thần', '▁thủ', '▁đức', '</s>']

----- Results from tokenizer.decode ----- 
<s> ▁hội ▁cư ▁dân ▁chung ▁cư ▁sen ▁hồng ▁- ▁chung ▁cư ▁lot us ▁sóng ▁thần ▁thủ ▁đức </s>



Now let's try PhoBert tokenizer. PhoBert tokenizer, unlike Envibert tokenizer, requires input to be word tokenized (using UnderTheSea library)

In [ ]:
from transformers import AutoTokenizer

In [ ]:
_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
inp = apply_vnmese_word_tokenize('hội cư dân chung cư sen hồng - chung cư lotus sóng thần thủ đức')
print(inp)

hội cư_dân chung_cư sen hồng - chung_cư lotus sóng_thần thủ_đức


In [ ]:
tokenizer_explain(inp,_tokenizer)

		------- Tokenizer Explained -------
----- Input -----
hội cư_dân chung_cư sen hồng - chung_cư lotus sóng_thần thủ_đức

----- Tokenized results ----- 
{'input_ids': [0, 1093, 1838, 1574, 3330, 2025, 31, 1574, 2029, 4885, 8554, 25625, 7344, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

----- Results from tokenizer.convert_ids_to_tokens -----
['<s>', 'hội', 'cư_dân', 'chung_cư', 'sen', 'hồng', '-', 'chung_cư', 'lo@@', 'tus', 'sóng_thần', 'thủ_@@', 'đức', '</s>']

----- Results from tokenizer.decode ----- 
<s> hội cư_dân chung_cư sen hồng - chung_cư lotus sóng_thần thủ_đức </s>



In [ ]:
#| export
def two_steps_tokenization_explain(inp, # Input sentence
                                   tokenizer, # Tokenizer (preferably from HuggingFace)
                                   content_tfms=[], # A list of text transformations
                                   aug_tfms=[], # A list of text augmentation 
                                  ):
    "Display results form each content transformation, then display results from tokenizer"
    print('\t\t------- Text Transformation Explained -------')
    print('----- Raw sentence -----')
    print(inp)
    print()
    print('----- Content Transformations (on both train and test) -----')
    content_tfms = val2iterable(content_tfms)
    for tfm in content_tfms:
        print_msg(callable_name(tfm),3)
        inp = tfm(inp)
        print(inp)
        print()
    print()
    print('----- Augmentations (on train only) -----')
    aug_tfms = val2iterable(aug_tfms)
    for tfm in aug_tfms:
        print_msg(callable_name(tfm),3)
        inp = tfm(inp)
        print(inp)
        print()
    print()
    tokenizer_explain(inp,tokenizer)

In [ ]:
show_doc(two_steps_tokenization_explain)

---

[source](https://github.com/anhquan0412/that-nlp-library/blob/main/that_nlp_library/text_main.py#L40){target="_blank" style="float:right; font-size:smaller"}

### two_steps_tokenization_explain

>      two_steps_tokenization_explain (inp, tokenizer, content_tfms=[],
>                                      aug_tfms=[])

Display results form each content transformation, then display results from tokenizer

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| inp |  |  | Input sentence |
| tokenizer |  |  | Tokenizer (preferably from HuggingFace) |
| content_tfms | list | [] | A list of text transformations |
| aug_tfms | list | [] | A list of text augmentation |

Let's load Phobert tokenizer one more time to test out this function

In [ ]:
_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from underthesea import text_normalize

`apply_vnmese_word_tokenize` also have an option to normalize text

In [ ]:
from functools import partial

In [ ]:
inp = 'Hội cư dân   chung cư sen hồng- chung cư    lotus sóng thần thủ đức. Thủ Đức là một huyện trực thuộc thành phố Hồ Chí Minh'
two_steps_tokenization_explain(inp,_tokenizer,content_tfms=[partial(apply_vnmese_word_tokenize,normalize_text=True)])

		------- Text Transformation Explained -------
----- Raw sentence -----
Hội cư dân   chung cư sen hồng- chung cư    lotus sóng thần thủ đức. Thủ Đức là một huyện trực thuộc thành phố Hồ Chí Minh

----- Content Transformations (on both train and test) -----
--- apply_vnmese_word_tokenize ---
Hội cư_dân chung_cư sen hồng - chung_cư lotus sóng_thần thủ_đức . Thủ_Đức là một huyện trực_thuộc thành_phố Hồ_Chí_Minh


----- Augmentations (on train only) -----

		------- Tokenizer Explained -------
----- Input -----
Hội cư_dân chung_cư sen hồng - chung_cư lotus sóng_thần thủ_đức . Thủ_Đức là một huyện trực_thuộc thành_phố Hồ_Chí_Minh

----- Tokenized results ----- 
{'input_ids': [0, 792, 1838, 1574, 3330, 2025, 31, 1574, 2029, 4885, 8554, 25625, 7344, 5, 5043, 8, 16, 149, 2850, 214, 784, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

----- Results from tokenizer.con

Let's add some text augmentations

In [ ]:
import unidecode

In [ ]:
# to remove vietnamese accent
remove_accent = lambda x: unidecode.unidecode(x)

If you want your function to be printed in with a different name:

In [ ]:
remove_accent.__name__ = 'Remove Vietnamese Accent'

In [ ]:
two_steps_tokenization_explain(inp,_tokenizer,
                               content_tfms=[partial(apply_vnmese_word_tokenize,normalize_text=True)],
                               aug_tfms=[remove_accent]
                              )

		------- Text Transformation Explained -------
----- Raw sentence -----
Hội cư dân   chung cư sen hồng- chung cư    lotus sóng thần thủ đức. Thủ Đức là một huyện trực thuộc thành phố Hồ Chí Minh

----- Content Transformations (on both train and test) -----
--- apply_vnmese_word_tokenize ---
Hội cư_dân chung_cư sen hồng - chung_cư lotus sóng_thần thủ_đức . Thủ_Đức là một huyện trực_thuộc thành_phố Hồ_Chí_Minh


----- Augmentations (on train only) -----
--- Remove Vietnamese Accent ---
Hoi cu_dan chung_cu sen hong - chung_cu lotus song_than thu_duc . Thu_Duc la mot huyen truc_thuoc thanh_pho Ho_Chi_Minh


		------- Tokenizer Explained -------
----- Input -----
Hoi cu_dan chung_cu sen hong - chung_cu lotus song_than thu_duc . Thu_Duc la mot huyen truc_thuoc thanh_pho Ho_Chi_Minh

----- Tokenized results ----- 
{'input_ids': [0, 3021, 1111, 56549, 17386, 22975, 13689, 3330, 27037, 31, 22975, 13689, 2029, 4885, 3227, 9380, 1510, 21605, 6190, 1894, 5, 5770, 4098, 1894, 2644, 3773, 1204, 189

You can even be creative with your augmentation functions; let's say you only want your augmentation to be applied 50% of the time:

In [ ]:
import random

In [ ]:
random.seed(2) # for reproducibility

In [ ]:
remove_accent = lambda x: unidecode.unidecode(x) if random.random()<0.5 else x
remove_accent.__name__ = 'Remove Vietnamese Accent with 0.5 prob'

In [ ]:
two_steps_tokenization_explain(inp,_tokenizer,
                               content_tfms=[partial(apply_vnmese_word_tokenize,normalize_text=True)],
                               aug_tfms=[remove_accent]
                              )

		------- Text Transformation Explained -------
----- Raw sentence -----
Hội cư dân   chung cư sen hồng- chung cư    lotus sóng thần thủ đức. Thủ Đức là một huyện trực thuộc thành phố Hồ Chí Minh

----- Content Transformations (on both train and test) -----
--- apply_vnmese_word_tokenize ---
Hội cư_dân chung_cư sen hồng - chung_cư lotus sóng_thần thủ_đức . Thủ_Đức là một huyện trực_thuộc thành_phố Hồ_Chí_Minh


----- Augmentations (on train only) -----
--- Remove Vietnamese Accent with 0.5 prob ---
Hội cư_dân chung_cư sen hồng - chung_cư lotus sóng_thần thủ_đức . Thủ_Đức là một huyện trực_thuộc thành_phố Hồ_Chí_Minh


		------- Tokenizer Explained -------
----- Input -----
Hội cư_dân chung_cư sen hồng - chung_cư lotus sóng_thần thủ_đức . Thủ_Đức là một huyện trực_thuộc thành_phố Hồ_Chí_Minh

----- Tokenized results ----- 
{'input_ids': [0, 792, 1838, 1574, 3330, 2025, 31, 1574, 2029, 4885, 8554, 25625, 7344, 5, 5043, 8, 16, 149, 2850, 214, 784, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 

There are more examples of interesting augmentation [here](https://anhquan0412.github.io/that-nlp-library/text_augmentation.html)

## Tokenize Function

In [ ]:
#| export
def tokenize_function(text,
                      tok,
                      max_length=None,
                      is_split_into_words=False,
                      return_tensors=None,
                      return_special_tokens_mask=False
                     ):
    if max_length is None:
        # pad to model's default max sequence length
        return tok(text, padding="max_length", 
                   truncation=True,
                   is_split_into_words=is_split_into_words,
                   return_tensors=return_tensors,
                   return_special_tokens_mask=return_special_tokens_mask
                  )
    if isinstance(max_length,int) and max_length>0:
        # pad to the largest length of the current batch, and start truncating at max_length
        return tok(text, padding=True, 
                   max_length=max_length,
                   truncation=True,
                   is_split_into_words=is_split_into_words,
                   return_tensors=return_tensors,
                   return_special_tokens_mask=return_special_tokens_mask
                  )
    
    # no padding (still truncate at model's default max sequence length)
    return tok(text, padding=False,
               truncation=True,
               is_split_into_words=is_split_into_words,
               return_tensors=return_tensors,
               return_special_tokens_mask=return_special_tokens_mask
              )

In [ ]:
show_doc(tokenize_function)

---

[source](https://github.com/anhquan0412/that-nlp-library/blob/main/that_nlp_library/text_main.py#L69){target="_blank" style="float:right; font-size:smaller"}

### tokenize_function

>      tokenize_function (text, tok, max_length=None, is_split_into_words=False,
>                         return_tensors=None)

Since I am processing Vietnamese text, I will use EnViBert's tokenizer. Envibert is a RoBERTa model for Vietnamese and English. This RoBERTa version is trained by using 100GB of text (50GB of Vietnamese and 50GB of English). For more information: [https://huggingface.co/nguyenvulebinh/envibert](https://huggingface.co/nguyenvulebinh/envibert)

In [ ]:
# https://huggingface.co/nguyenvulebinh/envibert
cache_dir=Path('./envibert_tokenizer')
_tokenizer = SourceFileLoader("envibert.tokenizer", 
                             str(cache_dir/'envibert_tokenizer.py')).load_module().RobertaTokenizer(cache_dir)

In [ ]:
tokenize_function('hội cư dân chung cư sen hồng - chung cư lotus sóng thần thủ đức',_tokenizer,max_length=512)

{'input_ids': [0, 227, 1033, 191, 664, 1033, 7366, 2615, 13, 664, 1033, 671, 1355, 2294, 993, 413, 2900, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenize_function(['hội cần mở thẻ tín dụng tại hà nội, đà nẵng, tp. hồ chí minh',
                   "biti's cao lãnh - đồng tháp"],
                  _tokenizer,max_length=512)

{'input_ids': [[0, 227, 256, 778, 2600, 1074, 144, 76, 5489, 613, 57339, 4820, 27666, 57339, 21422, 244, 872, 635, 841, 2], [0, 880, 592, 427, 162, 171, 906, 13, 122, 6553, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]}

You can change the tokenizer outputs' type, such as pytorch's tensor, tensorflow objects, or numpy array

In [ ]:
tokenize_function(['hội cần mở thẻ tín dụng tại hà nội, đà nẵng, tp. hồ chí minh',
                   "biti's cao lãnh - đồng tháp"],
                  _tokenizer,max_length=512,return_tensors='pt')

{'input_ids': tensor([[    0,   227,   256,   778,  2600,  1074,   144,    76,  5489,   613,
         57339,  4820, 27666, 57339, 21422,   244,   872,   635,   841,     2],
        [    0,   880,   592,   427,   162,   171,   906,    13,   122,  6553,
             2,     1,     1,     1,     1,     1,     1,     1,     1,     1]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}

In [ ]:
results = tokenize_function(['hội cần mở thẻ tín dụng tại hà nội, đà nẵng, tp. hồ chí minh',
                             "biti's cao lãnh - đồng tháp"],
                            _tokenizer,max_length=512)

In [ ]:
print(_tokenizer.convert_ids_to_tokens(results['input_ids'][0]))

['<s>', '▁hội', '▁cần', '▁mở', '▁thẻ', '▁tín', '▁dụng', '▁tại', '▁hà', '▁nội', ',', '▁đà', '▁nẵng', ',', '▁tp', '.', '▁hồ', '▁chí', '▁minh', '</s>']


You can change max_length (which allow truncation when sentence length is higher than max_length) 

In [ ]:
results = tokenize_function(['hội cần mở thẻ tín dụng tại hà nội, đà nẵng, tp. hồ chí minh',
                            "biti's cao lãnh - đồng tháp"],_tokenizer,
                            max_length=5)

In [ ]:
results

{'input_ids': [[0, 227, 256, 778, 2], [0, 880, 592, 427, 2]], 'token_type_ids': [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]}

## Metadatas Processing 

In [ ]:
#| export
def concat_metadatas(dset:dict, # HuggingFace Dataset
                     main_text, # Text feature name
                     metadatas, # Metadata (or a list of metadatas)
                     process_metas=True, # Whether apply simple metadata processing, i.e. space strip and lowercase
                     sep='.', # separator for contatenating to main_text
                     is_batched=True, # whether batching is applied
                    ):
    """
    Extract, process (optional) and concatenate metadatas to the front of text
    """
    results={main_text:dset[main_text]}
    for m in metadatas:
        m_data = dset[m]
        if process_metas:
            # just strip and lowercase
            m_data = [none2emptystr(v).strip().lower() for v in m_data] if is_batched else none2emptystr(m_data).strip().lower()
        results[m]=m_data
        if is_batched:
            results[main_text] = [f'{m_data[i]} {sep} {results[main_text][i]}' for i in range(len(m_data))]
        else:
            results[main_text] = f'{m_data} {sep} {results[main_text]}'
    return results

In [ ]:
show_doc(concat_metadatas)

---

[source](https://github.com/anhquan0412/that-nlp-library/blob/main/that_nlp_library/text_main.py#L86){target="_blank" style="float:right; font-size:smaller"}

### concat_metadatas

>      concat_metadatas (dset:dict, main_text, metadatas, process_metas=True,
>                        sep='.', is_batched=True)

Extract, process (optional) and concatenate metadatas to the front of text

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| dset | dict |  | HuggingFace Dataset |
| main_text |  |  | Text feature name |
| metadatas |  |  | Metadata (or a list of metadatas) |
| process_metas | bool | True | Whether apply simple metadata processing, i.e. space strip and lowercase |
| sep | str | . | separator for contatenating to main_text |
| is_batched | bool | True | whether batching is applied |

## Class TextDataController

In [ ]:
#| export
class TextDataController():
    def __init__(self,
                 inp, # HuggingFainpce Dataset or DatasetDict
                 main_text:str, # Name of the main text column
                 label_names=[], # Names of the label (dependent variable) columns
                 sup_types=[], # Type of supervised learning for each label name ('classification' or 'regression')
                 class_names_predefined=[], # List of names associated with the labels (same index order). Use empty list for regression
                 filter_dict={}, # A dictionary: {feature: filtering_function_for_that_feature}
                 label_tfm_dict={}, # A dictionary: {label_name: transform_function_for_that_label}
                 metadatas=[], # Names of the metadata columns
                 process_metas=True, # Whether to do simple text processing on the chosen metadatas
                 content_transformations=[], # A list of text transformations
                 val_ratio:int|float|None=0.2, # Ratio of data for validation set
                 stratify_cols=[], # Column(s) needed to do stratified shuffle split
                 upsampling_list={}, # A list of tuple. Each tuple: (feature,upsampling_function_based_on_the_feature)
                 content_augmentations=[], # A list of text augmentations
                 seed=None, # Random seed
                 batch_size=1000, # CPU batch size
                 num_proc=4, # Number of process for multiprocessing
                 cols_to_keep=None, # Columns to keep after all processings
                 verbose=True, # Whether to prdint processing information
                ):
            
        self.main_text = main_text
        
        self.label_names = val2iterable(label_names)
        self.sup_types = val2iterable(sup_types)
        self._check_sup_types()
        self.label_lists = class_names_predefined
        
        self.filter_dict = filter_dict
        self.label_tfm_dict = label_tfm_dict
        self.metadatas = val2iterable(metadatas)
        self.process_metas = process_metas
        
        self.content_tfms = val2iterable(content_transformations)
        
        self.val_ratio = val_ratio
        self.stratify_cols = val2iterable(stratify_cols)
        
        self.upsampling_list = upsampling_list
        self.aug_tfms = val2iterable(content_augmentations)
        
        self.seed = seed
        self.is_batched = batch_size>1
        self.batch_size = batch_size
        self.num_proc = num_proc
        self.cols_to_keep = cols_to_keep
        
        self.ddict_rest = DatasetDict()
        self.verbose = verbose
        self.verboseprint = print if verbose else lambda *a, **k: None
        
        if hasattr(inp,'keys'): # is datasetdict
            if 'train' in inp.keys(): 
                self.ddict_rest = inp
                self.dset = self.ddict_rest.pop('train')
            else:
                raise ValueError('The given DatasetDict has no "train" split')
        else: # is dataset
            self.dset = inp
        
        if isinstance(self.dset,IterableDataset):
            raise ValueError('TextDataController does not handle Iterable Dataset as input. Consider `TextDataControllerStreaming`')
        
        self._determine_val_key()
        self.all_cols = get_dset_col_names(self.dset)
        self._determine_multihead_multilabel()
        self._convert_regression_to_float()
        
        self._processed_call=False
            
    @classmethod
    def from_csv(cls,file_path,**kwargs):
        file_path = Path(file_path)
        ds = load_dataset(str(file_path.parent),
                                  data_files=file_path.name,
                                  split='train')
        return TextDataController(ds,**kwargs)
        
    
    @classmethod
    def from_df(cls,df,validate=True,**kwargs):
        if validate:
            check_input_validation(df)
        ds = Dataset.from_pandas(df)
        return TextDataController(ds,**kwargs)
    
    @classmethod
    def from_pickle(cls,
                    fname, # Name of the pickle file
                    parent='pickle_files' # Parent folder
                   ):
        return load_pickle(fname,parent=parent)

    def set_verbose(self,verbose):
        self.verbose = verbose
        self.verboseprint = print if verbose else lambda *a, **k: None
    
    def _convert_regression_to_float(self):
        if len(self.sup_types)==0: return
        # convert regression labels to float64
        reg_idxs = [i for i,v in enumerate(self.sup_types) if v=='regression']
        for i in reg_idxs:
            self.dset = self.dset.cast_column(self.label_names[i],Value("float64"))
            if self.val_key is not None:
                self.ddict_rest[self.val_key] = self.ddict_rest[self.val_key].cast_column(self.label_names[i],Value("float64"))
        
    def _check_sup_types(self):
        assert len(self.label_names)==len(self.sup_types), "The number of supervised learning declaration must equal to the number of label"
        assert len(set(self.sup_types) - set(['classification','regression']))==0, 'Accepted inputs for `sup_types` are `classification` and `regression`'
    
    def _determine_val_key(self):
        # determine validation split key
        self.val_key=None
        val_key = list(set(self.ddict_rest.keys()) & set(['val','validation','valid']))
        if len(val_key)>1: raise ValueError('Your DatasetDict has more than 1 validation split')
        if len(val_key)==1:
            val_key=val_key[0]
            self.val_key=val_key
            
    def _determine_multihead_multilabel(self):
        self.is_multilabel=False
        self.is_multihead=False
        if len(self.label_names)==0: return
        
        if len(self.label_names)>1:
            self.is_multihead=True
            
        # get label of first row
        first_label = self.dset[self.label_names[0]][0]
        if isinstance(first_label,(list,set,tuple)):
            # This is multi-label. Ignore self.label_names[1:]
            self.label_names = [self.label_names[0]]
            self.is_multihead=False
            self.is_multilabel=True
                     
    def validate_input(self):
        _df = self.dset.to_pandas()
        check_input_validation(_df)
    
    def save_as_pickles(self,
                        fname, # Name of the pickle file
                        parent='pickle_files', # Parent folder
                        drop_attributes=False # Whether to drop large-size attributes
                       ):
        if drop_attributes:
            if hasattr(self, 'main_ddict'):
                del self.main_ddict
            if hasattr(self, 'ddict_rest'):
                del self.ddict_rest
            if hasattr(self, 'aug_tfms'):
                del self.aug_tfms
        save_to_pickle(self,fname,parent=parent)
    
        
    def _check_validation_leaking(self):
        if self.val_ratio is None:
            return
        
        trn_txt = self.main_ddict['train'][self.main_text]
        val_txt = self.main_ddict['validation'][self.main_text]        
        val_txt_leaked = check_text_leaking(trn_txt,val_txt,verbose=self.verbose)
        
        if len(val_txt_leaked)==0: return
        
        # filter train dataset to get rid of leaks
        self.verboseprint('Filtering leaked data out of training set...')
        _func = partial(lambda_batch,
                        feature=self.main_text,
                        func=lambda x: x.strip().lower() not in val_txt_leaked,
                        is_batched=self.is_batched)
        self.main_ddict['train'] = hf_filter_dset(self.main_ddict['train'],_func,self.is_batched,self.batch_size,self.num_proc)   
        self.verboseprint('Done')
    
    def _process_metadatas(self,dset,ddict_rest=None):
        if len(self.metadatas):
            print_msg('Metadata Simple Processing & Concatenating to Main Content',verbose=self.verbose)
            map_func = partial(concat_metadatas,
                               main_text=self.main_text,
                               metadatas=self.metadatas,
                               process_metas=self.process_metas,
                               is_batched=self.is_batched)
            dset = hf_map_dset(dset,map_func,self.is_batched,self.batch_size,self.num_proc)
            if ddict_rest is not None:
                ddict_rest = hf_map_dset(ddict_rest,map_func,self.is_batched,self.batch_size,self.num_proc)
            self.verboseprint('Done')
        return dset if ddict_rest is None else (dset,ddict_rest)
    
    def _do_label_transformation(self):
        if len(self.label_names)==0 or len(self.label_tfm_dict)==0: return
        print_msg('Label Transformation',20,verbose=self.verbose)
        for f,tfm in self.label_tfm_dict.items():
            if f in self.label_names:
                _func = partial(lambda_map_batch,
                                feature=f,
                                func=tfm,
                                is_batched=self.is_batched
                               )                
                self.dset = hf_map_dset(self.dset,_func,self.is_batched,self.batch_size,self.num_proc)
                if self.val_key is not None:
                    self.ddict_rest[self.val_key] = hf_map_dset(self.ddict_rest[self.val_key],
                                                                _func,
                                                                self.is_batched,
                                                                self.batch_size,
                                                                self.num_proc)
        self.verboseprint('Done')
        
    def _create_label_mapping_func(self,encoder_classes):
        if self.is_multihead:
            label2idxs = [{v:i for i,v in enumerate(l_classes)} for l_classes in encoder_classes]
            _func = lambda inp: {'label': [[label2idxs[i][v] if len(label2idxs[i]) else v for i,v in enumerate(vs)] \
                                           for vs in zip(*[inp[l] for l in self.label_names])] if self.is_batched \
                                 else [label2idxs[i][v] if len(label2idxs[i]) else v for i,v in enumerate([inp[l] for l in self.label_names])]
                                }
            
        else: # single-head
            if self.sup_types[0]=='regression':
                _func1 = lambda x: x
            else:
                label2idx = {v:i for i,v in enumerate(encoder_classes[0])}
                _func1 = lambda x: label2idx[x]
                
            _func = partial(lambda_map_batch,
                           feature=self.label_names[0],
                           func=_func1,
                           output_feature='label',
                           is_batched=self.is_batched)
        return _func
        
    def _encode_labels(self):
        if len(self.label_names)==0: return
        print_msg('Label Encoding',verbose=self.verbose)
        
        if len(self.label_lists) and not isinstance(self.label_lists[0],list):
            self.label_lists = [self.label_lists]
                    
        encoder_classes=[]
        if not self.is_multilabel:
            for idx,l in enumerate(self.label_names):
                if self.sup_types[idx]=='regression':
                    l_classes=[]
                else:
                    # classification
                    if len(self.label_lists)==0:
                        l_encoder = LabelEncoder()
                        _ = l_encoder.fit(self.dset[l])
                        l_classes = list(l_encoder.classes_)
                    else:
                        l_classes = sorted(list(self.label_lists[idx]))
                encoder_classes.append(l_classes)
            
            _func = self._create_label_mapping_func(encoder_classes)
                
            self.dset = hf_map_dset(self.dset,_func,self.is_batched,self.batch_size,self.num_proc)
            if self.val_key is not None:
                self.ddict_rest[self.val_key] = hf_map_dset(self.ddict_rest[self.val_key],_func,
                                                       self.is_batched,self.batch_size,self.num_proc)
                    
        else:
            # For MultiLabel, we transform the label itself to one-hot (or actually, few-hot)
            if len(self.label_lists)==0:
                l_encoder = MultiLabelBinarizer()
                _ = l_encoder.fit(self.dset[self.label_names[0]])
                l_classes = list(l_encoder.classes_)
            else:
                l_classes = sorted(list(self.label_lists[0]))
            
            encoder_classes.append(l_classes)
            
            l_encoder = MultiLabelBinarizer(classes=encoder_classes[0])
            _ = l_encoder.fit(None)
            _func = partial(lambda_map_batch,
                            feature=self.label_names[0],
                            func=lambda x: l_encoder.transform(x),
                            output_feature='label',
                            is_batched=self.is_batched,
                            is_func_batched=True)
            self.dset = hf_map_dset(self.dset,_func,self.is_batched,self.batch_size,self.num_proc)                                                 
            
            if self.val_key is not None:
                self.ddict_rest[self.val_key] = hf_map_dset(self.ddict_rest[self.val_key],_func,
                                                       self.is_batched,self.batch_size,self.num_proc)
            
        self.label_lists = encoder_classes
        self.verboseprint('Done')
        
    
    def _train_test_split(self):
        print_msg('Train Test Split',20,verbose=self.verbose)
        if self.val_key is not None: # val split exists
            self.verboseprint('Validation split already exists')
            self.main_ddict=DatasetDict({'train':self.dset,
                                         'validation':self.ddict_rest.pop(self.val_key)})
            
    
        elif self.val_ratio is None: # use all data
            self.verboseprint('No validation split defined')
            self.main_ddict=DatasetDict({'train':self.dset})
            
        elif (isinstance(self.val_ratio,float) or isinstance(self.val_ratio,int)) and len(self.stratify_cols)==0:
            self.verboseprint('Validation split based on val_ratio')
            # train val split
            self.main_ddict = self.dset.train_test_split(test_size=self.val_ratio,shuffle=True,seed=self.seed)
            self.main_ddict['validation']=self.main_ddict['test']
            del self.main_ddict['test']
        
        else: # val_ratio split with stratifying             
            if self.is_multilabel and self.label_names[0] in self.stratify_cols:
                raise ValueError('For MultiLabel classification, you cannot choose the label as your stratified column')
            self.verboseprint('Validation split based on val_ratio, with stratifying')
            # Create a new feature 'stratified', which is a concatenation of values in stratify_cols
            if self.is_batched:
                stratified_creation = lambda x: {'stratified':
                                     ['_'.join(list(map(str,[x[v][i] for v in self.stratify_cols]))) 
                                      for i in range(len(x[self.stratify_cols[0]]))]}
            else:
                stratified_creation = lambda x: {'stratified':
                                     '_'.join(list(map(str,[x[v] for v in self.stratify_cols]))) 
                                      }
            self.dset = self.dset.map(stratified_creation,
                                      batched=self.is_batched,
                                      batch_size=self.batch_size,
                                      num_proc=self.num_proc)
            
            self.dset=self.dset.class_encode_column("stratified")
            # train val split
            self.main_ddict = self.dset.train_test_split(test_size=self.val_ratio,
                                                         shuffle=True,seed=self.seed,
                                                        stratify_by_column='stratified')
            self.main_ddict['validation']=self.main_ddict['test']
            del self.main_ddict['test']
            self.main_ddict=self.main_ddict.remove_columns(['stratified'])
            
        
        del self.dset
        self.verboseprint('Done')
    
    def _simplify_ddict(self):
        print_msg('Dropping unused features',20,verbose=self.verbose)
        if self.cols_to_keep is None:
            self.cols_to_keep= [self.main_text] + self.metadatas + self.label_names
            
        cols_to_remove = set(self.all_cols) - set(self.cols_to_keep)
        self.main_ddict['train']=self.main_ddict['train'].remove_columns(list(cols_to_remove))
        if 'validation' in self.main_ddict.keys():
            self.main_ddict['validation']=self.main_ddict['validation'].remove_columns(list(cols_to_remove))
        self.verboseprint('Done')
    
    def _do_transformation(self,dset,ddict_rest=None):
        if len(self.content_tfms):
            print_msg('Text Transformation',20,verbose=self.verbose)
            for tfm in self.content_tfms:
                print_msg(callable_name(tfm),verbose=self.verbose)
                _func = partial(lambda_map_batch,
                               feature=self.main_text,
                               func=tfm,
                               is_batched=self.is_batched)
                dset = hf_map_dset(dset,_func,self.is_batched,self.batch_size,self.num_proc)
                if ddict_rest is not None:
                    ddict_rest = hf_map_dset(ddict_rest,_func,self.is_batched,self.batch_size,self.num_proc)
            self.verboseprint('Done')
        return dset if ddict_rest is None else (dset,ddict_rest)
 
    def _do_filtering(self,dset,ddict_rest=None):
        if len(self.filter_dict):
            print_msg('Data Filtering',20,verbose=self.verbose)
            col_names = get_dset_col_names(dset)
            for f,tfm in self.filter_dict.items():
                if f in col_names:
                    print_msg(f'Do {callable_name(tfm)} on {f}',verbose=self.verbose)
                    _func = partial(lambda_batch,
                                    feature=f,
                                    func=tfm,
                                    is_batched=self.is_batched)
                    dset = hf_filter_dset(dset,_func,self.is_batched,self.batch_size,self.num_proc)
                if ddict_rest is not None: # assuming ddict_rest has the column to filter, always
                    ddict_rest = hf_filter_dset(ddict_rest,_func,self.is_batched,self.batch_size,self.num_proc)
            self.verboseprint('Done')
        return dset if ddict_rest is None else (dset,ddict_rest)
    
    def _upsampling(self):
        if len(self.upsampling_list):
            print_msg('Upsampling data',20,verbose=self.verbose)
            results=[]
            for f,tfm in self.upsampling_list:
                print_msg(f'Do {callable_name(tfm)} on {f}',verbose=self.verbose)
                _func = partial(lambda_batch,
                                feature=f,
                                func=tfm,
                                is_batched=self.is_batched)
                new_dset = hf_filter_dset(self.main_ddict['train'],_func,self.is_batched,self.batch_size,self.num_proc)
                results.append(new_dset)
            # slow concatenation for iterable dataset    
            self.main_ddict['train'] = concatenate_datasets(results+[self.main_ddict['train']])
            self.verboseprint('Done')
      
    def _do_augmentation(self):
        if len(self.aug_tfms):
            print_msg('Text Augmentation',20,verbose=self.verbose)
            if self.seed:
                seed_everything(self.seed)   
            for tfm in self.aug_tfms:
                print_msg(callable_name(tfm),verbose=self.verbose)
                bs = self.batch_size
                is_func_batched=False
                num_proc = self.num_proc
                is_batched = self.is_batched
                if hasattr(tfm, "run_on_gpu") and getattr(tfm,'run_on_gpu')==True:
                    bs = 32 if not hasattr(tfm, "batch_size") else getattr(tfm,'batch_size')
                    is_func_batched=True
                    is_batched=True
                    num_proc=1

                _func = partial(lambda_map_batch,
                               feature=self.main_text,
                               func=tfm,
                               is_batched=is_batched,
                               is_func_batched=is_func_batched
                               )
                self.main_ddict['train'] = hf_map_dset(self.main_ddict['train'],_func,
                                                          is_batched=is_batched,
                                                          batch_size=bs,
                                                          num_proc=num_proc
                                                         )


            self.verboseprint('Done')
        

            
    def _do_train_shuffling(self):
        print_msg('Shuffling and flattening train set',20,verbose=self.verbose)
        self.main_ddict['train'] = self.main_ddict['train'].shuffle(seed=self.seed).flatten_indices(num_proc = self.num_proc)
        self.verboseprint('Done')
        
    def do_all_preprocessing(self,
                             shuffle_trn=True # To shuffle the train set before tokenization
                            ):
        if self._processed_call:
            warnings.warn('Your dataset has already been processed. Returning the previous processed DatasetDict...')
            return self.main_ddict
            
        print_msg('Start Main Text Processing',20,verbose=self.verbose)
        
        # Filtering
        self.dset,self.ddict_rest = self._do_filtering(self.dset,self.ddict_rest)
        
        # Process metadatas
        self.dset,self.ddict_rest = self._process_metadatas(self.dset,self.ddict_rest)
        
        # Label transformation
        self._do_label_transformation()
        
        # Process labels
        self._encode_labels()
        
        # Content transformation
        self.dset,self.ddict_rest = self._do_transformation(self.dset,self.ddict_rest)
         
        # Train Test Split.
        ### self.main_ddict is created here
        self._train_test_split()
        
        # Dropping unused columns
        self._simplify_ddict()
        
        # Check validation leaking
        self._check_validation_leaking()
        
        ### The rest of these functions applies only to the train dataset
        # Upsampling
        self._upsampling()
        
        # Augmentation
        self._do_augmentation()
        
        # Shuffle train
        if shuffle_trn:
            self._do_train_shuffling()
        
        self._processed_call=True
        
        return self.main_ddict
    
        
    def do_tokenization(self,
                        tokenizer, # Tokenizer (preferably from HuggingFace)
                        max_length=None, # pad to model's allowed max length (default is max_sequence_length). Use -1 for no padding at all
                        trn_size=None, # The number of training data to be tokenized
                       ):
        print_msg('Tokenization',20,verbose=self.verbose)
        self.tokenizer = tokenizer
        self.max_length = max_length
        tok_func = partial(tokenize_function,tok=tokenizer,max_length=max_length)
        _func = partial(lambda_map_batch,
                        feature=self.main_text,
                        func=tok_func,
                        output_feature=None,
                        is_batched=self.is_batched)
        
        if trn_size is not None:
            if isinstance(trn_size,float):
                num_shard = int(1/trn_size)
            else: # int
                trn_len=len(self.main_ddict['train'])
                num_shard = trn_len//trn_size
            self.main_ddict['train'] = self.main_ddict['train'].shard(num_shard,0)
        
        for k in self.main_ddict.keys():
            self.main_ddict[k] = hf_map_dset(self.main_ddict[k],_func,self.is_batched,self.batch_size,self.num_proc)

        self.verboseprint('Done')
        return self.main_ddict
        
    def process_and_tokenize(self,
                             tokenizer, # Tokenizer (preferably from HuggingFace)
                             max_length=None, # pad to model's allowed max length (default is max_sequence_length)
                             trn_size=None, # The number of training data to be tokenized
                             shuffle_trn=True, # To shuffle the train set before tokenization
                            ):
        """
        This will perform `do_all_processing` then `do_tokenization`
        """
        _ = self.do_all_preprocessing(shuffle_trn)
        _ = self.do_tokenization(tokenizer,max_length,trn_size)
        
    
    def set_data_collator(self,data_collator):
        self.data_collator = data_collator
        
    
    def prepare_test_dataset_from_csv(self,
                                      file_path, # path to csv file
                                      do_filtering=False # whether to perform data filtering on this test set
                                     ):
        file_path = Path(file_path)
        ds = load_dataset(str(file_path.parent),
                          data_files=file_path.name,
                          split='train')
        return self.prepare_test_dataset(ds,do_filtering)
    
    def prepare_test_dataset_from_df(self,
                                     df, # Pandas Dataframe
                                     validate=True, # whether to perform input data validation
                                     do_filtering=False # whether to perform data filtering on this test set 
                                    ):
        if validate:
            check_input_validation(df)
        ds = Dataset.from_pandas(df)
        return self.prepare_test_dataset(ds,do_filtering)
    
    def prepare_test_dataset_from_raws(self,
                                       content, # Either a single sentence, list of sentence or a dictionary with keys are metadata columns and values are list
                                      ):
        if len(self.metadatas) and not isinstance(content,dict):
            raise ValueError(f'There is/are metadatas in the preprocessing step. Please include a dictionary including these keys for metadatas: {self.metadatas}, and texture content: {self.main_text}')
            
        _dic = {self.main_text:[content]} if isinstance(content,str) else content
        for k in _dic.keys():
            _dic[k] = val2iterable(_dic[k])
        
        test_dict = Dataset.from_dict(_dic)
        
        # set num_proc to 1 for small data processing
        _tmp = self.num_proc
        self.num_proc=1
        results = self.prepare_test_dataset(test_dict,do_filtering=False)
        self.num_proc = _tmp
        return results
    
    def prepare_test_dataset(self,
                             test_dset, # The HuggingFace Dataset as Test set
                             do_filtering=False, # whether to perform data filtering on this test set
                            ):
        test_cols = set(get_dset_col_names(test_dset))
        label_names_set = set(self.label_names)
        test_cols = test_cols - label_names_set
        missing_cols = set(self.cols_to_keep) - label_names_set - test_cols
        if len(missing_cols):
            raise ValueError(f'Test set does not have these columns required for preprocessings: {missing_cols}')
            
        print_msg('Start Test Set Transformation',20,verbose=self.verbose)

        # Filtering
        if do_filtering:
            test_dset = self._do_filtering(test_dset)
        
        # Process metadatas
        test_dset = self._process_metadatas(test_dset)
        
        # Content transformation
        test_dset = self._do_transformation(test_dset)
        
        # Drop unused columns
        cols_to_remove = test_cols - set(self.cols_to_keep)
        test_dset=test_dset.remove_columns(list(cols_to_remove))
        
        # Tokenization
        print_msg('Tokenization',20,verbose=self.verbose)
        _func = partial(lambda_map_batch,
                        feature=self.main_text,
                        func=partial(tokenize_function,tok=self.tokenizer,max_length=self.max_length),
                        output_feature=None,
                        is_batched=self.is_batched)
        test_dset = hf_map_dset(test_dset,_func,self.is_batched,self.batch_size,self.num_proc)
        
        self.verboseprint('Done')
        return test_dset


In [ ]:
show_doc(TextDataController)

---

[source](https://github.com/anhquan0412/that-nlp-library/blob/main/that_nlp_library/text_main.py#L110){target="_blank" style="float:right; font-size:smaller"}

### TextDataController

>      TextDataController (inp, main_text:str, label_names=[], sup_types=[],
>                          class_names_predefined=[], filter_dict={},
>                          label_tfm_dict={}, metadatas=[], process_metas=True,
>                          content_transformations=[],
>                          val_ratio:int|float|None=0.2, stratify_cols=[],
>                          upsampling_list={}, content_augmentations=[],
>                          seed=None, batch_size=1000, num_proc=4,
>                          cols_to_keep=None, verbose=True)

Initialize self.  See help(type(self)) for accurate signature.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| inp |  |  | HuggingFainpce Dataset or DatasetDict |
| main_text | str |  | Name of the main text column |
| label_names | list | [] | Names of the label (dependent variable) columns |
| sup_types | list | [] | Type of supervised learning for each label name ('classification' or 'regression') |
| class_names_predefined | list | [] | List of names associated with the labels (same index order). Use empty list for regression |
| filter_dict | dict | {} | A dictionary: {feature: filtering_function_for_that_feature} |
| label_tfm_dict | dict | {} | A dictionary: {label_name: transform_function_for_that_label} |
| metadatas | list | [] | Names of the metadata columns |
| process_metas | bool | True | Whether to do simple text processing on the chosen metadatas |
| content_transformations | list | [] | A list of text transformations |
| val_ratio | int \| float \| None | 0.2 | Ratio of data for validation set |
| stratify_cols | list | [] | Column(s) needed to do stratified shuffle split |
| upsampling_list | dict | {} | A list of tuple. Each tuple: (feature,upsampling_function_based_on_the_feature) |
| content_augmentations | list | [] | A list of text augmentations |
| seed | NoneType | None | Random seed |
| batch_size | int | 1000 | CPU batch size |
| num_proc | int | 4 | Number of process for multiprocessing |
| cols_to_keep | NoneType | None | Columns to keep after all processings |
| verbose | bool | True | Whether to print processing information |

In [ ]:
show_doc(TextDataController.do_all_preprocessing)

---

[source](https://github.com/anhquan0412/that-nlp-library/blob/main/that_nlp_library/text_main.py#L494){target="_blank" style="float:right; font-size:smaller"}

### TextDataController.do_all_preprocessing

>      TextDataController.do_all_preprocessing (shuffle_trn=True)

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| shuffle_trn | bool | True | To shuffle the train set before tokenization |

In [ ]:
show_doc(TextDataController.do_tokenization)

---

[source](https://github.com/anhquan0412/that-nlp-library/blob/main/that_nlp_library/text_main.py#L541){target="_blank" style="float:right; font-size:smaller"}

### TextDataController.do_tokenization

>      TextDataController.do_tokenization (tokenizer, max_length=None,
>                                          trn_size=None)

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| tokenizer |  |  | Tokenizer (preferably from HuggingFace) |
| max_length | NoneType | None | pad to model's allowed max length (default is max_sequence_length) |
| trn_size | NoneType | None | The number of training data to be tokenized |

In [ ]:
show_doc(TextDataController.process_and_tokenize)

---

[source](https://github.com/anhquan0412/that-nlp-library/blob/main/that_nlp_library/text_main.py#L570){target="_blank" style="float:right; font-size:smaller"}

### TextDataController.process_and_tokenize

>      TextDataController.process_and_tokenize (tokenizer, max_length=None,
>                                               trn_size=None, shuffle_trn=True)

This will perform `do_all_processing` then `do_tokenization`

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| tokenizer |  |  | Tokenizer (preferably from HuggingFace) |
| max_length | NoneType | None | pad to model's allowed max length (default is max_sequence_length) |
| trn_size | NoneType | None | The number of training data to be tokenized |
| shuffle_trn | bool | True | To shuffle the train set before tokenization |

## Load data + Basic use case

In [ ]:
show_doc(TextDataController.from_csv)

---

[source](https://github.com/anhquan0412/that-nlp-library/blob/main/that_nlp_library/text_main.py#L171){target="_blank" style="float:right; font-size:smaller"}

### TextDataController.from_csv

>      TextDataController.from_csv (file_path, **kwargs)

In [ ]:
show_doc(TextDataController.from_df)

---

[source](https://github.com/anhquan0412/that-nlp-library/blob/main/that_nlp_library/text_main.py#L180){target="_blank" style="float:right; font-size:smaller"}

### TextDataController.from_df

>      TextDataController.from_df (df, validate=True, **kwargs)

You can create a `TextDataController` from a csv, pandas DataFrame, or directly from a HuggingFace dataset object. Currently, `TextDataController` is designed for text classification and text regression, as we will explore in this documentation

We will load a sample data to prepare for a classification task: which `Department Name` a comment (`Review Text`) belongs to 

Dataset source: https://www.kaggle.com/datasets/kavita5/review_ecommerce

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('sample_data/Womens_Clothing_Reviews.csv',encoding='utf-8-sig')

In [ ]:
df.shape

(23486, 10)

In [ ]:
df.sample(5) 

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
14279,879,42,"Great, flattering, comfy",I run between an xs and s in most retailer shi...,5,1,0,General Petite,Tops,Knits
21591,1083,41,NaN,I wore this dress for an after party the day o...,5,1,19,General,Dresses,Dresses
14291,1073,37,Sad this couldn't come home with me....,So i really loved this dress. i am tall (6ft) ...,2,0,7,General Petite,Dresses,Dresses
12284,875,28,NaN,Fit is oversized but intentional. flattering o...,5,1,0,General,Tops,Knits
19322,1008,43,Gorgeous!,I love this skirt!! it slips seamlessly from d...,5,1,0,General Petite,Bottoms,Skirts


You can create a `TextDataController` from a dataframe. This also provides a quick input validation check (NaN check and Duplication check)

In [ ]:
tdc = TextDataController.from_df(df,
                                 main_text='Review Text',
                                 sup_types='classification',
                                 label_names='Department Name',
                                )

- Input Validation Precheck -
Data contains missing values!
-----> List of columns and the number of missing values for each
Title              3810
Review Text         845
Division Name        14
Department Name      14
Class Name           14
dtype: int64
Data contains duplicated values!
-----> Number of duplications: 21 rows


You can also create a `TextDataController` directly from the csv file. The good thing about using HuggingFace Dataset as the main backend of the TextDataController is that you can utilize lots of its useful functionality, such as caching

In [ ]:
tdc = TextDataController.from_csv('sample_data/Womens_Clothing_Reviews.csv',
                                  main_text='Review Text',
                                  sup_types='classification',
                                  label_names='Department Name',
                                 )

You can also create a `TextDataController` from a HuggingFace Dataset

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
dset

Dataset({
    features: ['Clothing ID', 'Age', 'Title', 'Review Text', 'Rating', 'Recommended IND', 'Positive Feedback Count', 'Division Name', 'Department Name', 'Class Name'],
    num_rows: 23486
})

In [ ]:
tdc = TextDataController(dset,
                         main_text='Review Text',
                         sup_types='classification',
                         label_names='Department Name',
                         seed=42
                        )

In the "Input Validation Precheck" above, we notice that our dataset has missing values in the text field and the label field. For now, let's load the data as a Pandas' DataFrame, perform some cleaning, and create our `TextDataController`

In [ ]:
df = pd.read_csv('sample_data/Womens_Clothing_Reviews.csv',encoding='utf-8-sig')

In [ ]:
df = df[(~df['Review Text'].isna()) & (~df['Department Name'].isna())].reset_index(drop=True)

In [ ]:
tdc = TextDataController.from_df(df,
                                 main_text='Review Text',
                                 sup_types='classification',
                                 label_names='Department Name',
                                )

- Input Validation Precheck -
Data contains missing values!
-----> List of columns and the number of missing values for each
Title    2966
dtype: int64
Data contains duplicated values!
-----> Number of duplications: 1 rows


At this point you can start perform 2 important steps on your data

1. Text preprocessings, Label Encoding, Train/Validation Split
2. Tokenization

We haven't provided any preprocessings to the `TextDataController`; we will see more on how to use preprocessings (step by step) as we progress

In [ ]:
ddict = tdc.do_all_preprocessing(shuffle_trn=True)

-------------------- Start Main Text Processing --------------------
----- Label Encoding -----


Map (num_proc=4):   0%|          | 0/22628 [00:00<?, ? examples/s]

Done
-------------------- Train Test Split --------------------
Validation split based on val_ratio
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 2, which is 0.01% of training set
Filtering leaked data out of training set...


Filter (num_proc=4):   0%|          | 0/18102 [00:00<?, ? examples/s]

Done
-------------------- Shuffling and flattening train set --------------------


Flattening the indices (num_proc=4):   0%|          | 0/18100 [00:00<?, ? examples/s]

Done


In [ ]:
ddict

DatasetDict({
    train: Dataset({
        features: ['Review Text', 'Department Name', 'label'],
        num_rows: 18100
    })
    validation: Dataset({
        features: ['Review Text', 'Department Name', 'label'],
        num_rows: 4526
    })
})

Our DatasetDict now has two split: train and validation. Note that train split is now IterableDataset, for processing efficiency

In [ ]:
ddict['train'][:3]

{'Review Text': ['I wish i had read the other reviews before purchasing this product. label says that you can wash and dry normally. however, upon washing, the outer layer shrunk and the inner lining did not. the robe will now not lay flat; the fabric pulls at the edges, and results in the inner lining being revealed when the robe is tied shut. extremely disappointed. for a $100 garment, i expected much more. i wish i could return this.',
  'Tried these on several times in store and never bought them, but was able to get them with a 20% email- seemed like a better price for the product. the color is rust and a little odd for my wardrobe - wish they also came in black. love the style and they are very comfortable, perfect for late summer- can wear on errands or dress them up a little with shoes/accessories for casual dinner out. happy i finally purchased them!',
  'I ordered this in the peach color and it is a beautiful pale color. the underside of the cuff has a pretty pattern on it, s

In [ ]:
ddict['validation'][:3]

{'Review Text': ['This is such a comfy dress and recommend it to anyone. pairs amazing with leather boots and hat',
  'This sweater is a hit. the color is fantastic and the shoulder cutouts add a feminine and unique touch. looks amazing paired with booties, a statement necklace and jeans. definitely affordable too! highly recommend',
  "This dress is really beautiful. i was looking for something to wear to a summer 6 o'clock wedding, and this dress will work perfectly. the fabric is smooth and has a tiny bit of sheen to it which makes it look a little nicer and more formal. however, you can easily dress it down. my only issue was the length. i'm 5'4 and this dress was way too long. definitely need to take off about 5 or 6 inches! i also had to size up. i could have tried the petite, but it fit so well everywhere except length,"],
 'Department Name': ['Intimate', 'Tops', 'Dresses'],
 'label': [2, 4, 1]}

Now we can start with the tokenization

In [ ]:
from transformers import RobertaTokenizer

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [ ]:
ddict = tdc.do_tokenization(tokenizer,max_length=512)

-------------------- Tokenization --------------------


Map (num_proc=4):   0%|          | 0/18100 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4526 [00:00<?, ? examples/s]

Done


In [ ]:
ddict

DatasetDict({
    train: Dataset({
        features: ['Review Text', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 18100
    })
    validation: Dataset({
        features: ['Review Text', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 4526
    })
})

In [ ]:
print(ddict['train'][0]['input_ids'][:150])

[0, 100, 2813, 939, 56, 1166, 5, 97, 6173, 137, 4906, 42, 1152, 4, 6929, 161, 14, 47, 64, 10397, 8, 3841, 6329, 4, 959, 6, 2115, 14784, 6, 5, 15705, 10490, 28704, 8, 5, 8725, 14187, 222, 45, 4, 5, 36393, 40, 122, 45, 4477, 3269, 131, 5, 10199, 16427, 23, 5, 15716, 6, 8, 775, 11, 5, 8725, 14187, 145, 1487, 77, 5, 36393, 16, 3016, 2572, 4, 2778, 5779, 4, 13, 10, 68, 1866, 25916, 6, 939, 421, 203, 55, 4, 939, 2813, 939, 115, 671, 42, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
print(ddict['validation'][0]['input_ids'][:150])

[0, 713, 16, 215, 10, 3137, 24382, 3588, 8, 5940, 24, 7, 1268, 4, 15029, 2770, 19, 9219, 10317, 8, 3988, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


You can combine Text Processing and Tokenization with 1 method call

In [ ]:
tdc = TextDataController.from_df(df,
                                 main_text='Review Text',
                                 sup_types='classification',
                                 label_names='Department Name'
                                )

- Input Validation Precheck -
Data contains missing values!
-----> List of columns and the number of missing values for each
Title    2966
dtype: int64
Data contains duplicated values!
-----> Number of duplications: 1 rows


In [ ]:
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)

-------------------- Start Main Text Processing --------------------
----- Label Encoding -----


Map (num_proc=4):   0%|          | 0/22628 [00:00<?, ? examples/s]

Done
-------------------- Train Test Split --------------------
Validation split based on val_ratio
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 0, which is 0.00% of training set
-------------------- Shuffling and flattening train set --------------------


Flattening the indices (num_proc=4):   0%|          | 0/18102 [00:00<?, ? examples/s]

Done
-------------------- Tokenization --------------------


Map (num_proc=4):   0%|          | 0/18102 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4526 [00:00<?, ? examples/s]

Done


You can access the DatasetDict from the instance variable `main_ddict`

In [ ]:
tdc.main_ddict

DatasetDict({
    train: Dataset({
        features: ['Review Text', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 18102
    })
    validation: Dataset({
        features: ['Review Text', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 4526
    })
})

This DatasetDict is ready to be put into any HuggingFace text model.

## Filtering

This preprocessing step allow you to filter out certain values of a certain column in your dataset. Let's say I want to filter out any None value in the column 'Review Text'

In [ ]:
df = pd.read_csv('sample_data/Womens_Clothing_Reviews.csv',encoding='utf-8-sig')
df[(~df['Review Text'].isna())].isna().sum()

Clothing ID                   0
Age                           0
Title                      2966
Review Text                   0
Rating                        0
Recommended IND               0
Positive Feedback Count       0
Division Name                13
Department Name              13
Class Name                   13
dtype: int64

We will provide a dictionary containing the name of the column and the filtering function to apply on that column. Note that **the filtering function will receive an item from the column, and the function should return a boolean**

In [ ]:
tdc = TextDataController.from_df(df,
                                 main_text='Review Text',
                                 sup_types='classification',
                                 label_names='Department Name',
                                 filter_dict={'Review Text': lambda x: x is not None},
                                 seed=42
                                )

- Input Validation Precheck -
Data contains missing values!
-----> List of columns and the number of missing values for each
Title              3810
Review Text         845
Division Name        14
Department Name      14
Class Name           14
dtype: int64
Data contains duplicated values!
-----> Number of duplications: 21 rows


In [ ]:
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)

-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----


Filter (num_proc=4):   0%|          | 0/23486 [00:00<?, ? examples/s]

Done
----- Label Encoding -----


Map (num_proc=4):   0%|          | 0/22641 [00:00<?, ? examples/s]

Done
-------------------- Train Test Split --------------------
Validation split based on val_ratio
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 1, which is 0.01% of training set
Filtering leaked data out of training set...


...

Flattening the indices (num_proc=4):   0%|          | 0/18111 [00:00<?, ? examples/s]

Done
-------------------- Tokenization --------------------


Map (num_proc=4):   0%|          | 0/18111 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4529 [00:00<?, ? examples/s]

Done


In [ ]:
tdc.main_ddict

DatasetDict({
    train: Dataset({
        features: ['Review Text', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 18111
    })
    validation: Dataset({
        features: ['Review Text', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 4529
    })
})

Let's check if we have filtered out all NaN/None value

In [ ]:
for i in tdc.main_ddict['train']['Review Text']:
    assert i is not None
for i in tdc.main_ddict['validation']['Review Text']:
    assert i is not None

We can even add multiple filtering functions. Remember from our precheck, there are also None values in our label 'Department Name'. While we are at it, let's filter out any rating that is less than 3 (just to showcase what our filtering can do)

In [ ]:
df.Rating.value_counts()

Rating
5    13131
4     5077
3     2871
2     1565
1      842
Name: count, dtype: int64

Note that `TextDataController` will only keep the text, the labels and the metadatas columns; any other column will be dropped. To keep the 'Rating', we need to define the `cols_to_keep` argument

In [ ]:
df = pd.read_csv('sample_data/Womens_Clothing_Reviews.csv',encoding='utf-8-sig')
tdc = TextDataController.from_df(df,
                                 main_text='Review Text',
                                 sup_types='classification',
                                 label_names='Department Name',
                                 filter_dict={'Review Text': lambda x: x is not None,
                                              'Department Name': lambda x: x is not None,
                                              'Rating': lambda x: x>=3
                                             },
                                 cols_to_keep=['Review Text','Rating','Department Name'],
                                 seed=42
                                )

- Input Validation Precheck -
Data contains missing values!
-----> List of columns and the number of missing values for each
Title              3810
Review Text         845
Division Name        14
Department Name      14
Class Name           14
dtype: int64
Data contains duplicated values!
-----> Number of duplications: 21 rows


In [ ]:
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)

-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----


Filter (num_proc=4):   0%|          | 0/23486 [00:00<?, ? examples/s]

----- Do <lambda> on Department Name -----


Filter (num_proc=4):   0%|          | 0/22641 [00:00<?, ? examples/s]

----- Do <lambda> on Rating -----


...

Flattening the indices (num_proc=4):   0%|          | 0/16205 [00:00<?, ? examples/s]

Done
-------------------- Tokenization --------------------


Map (num_proc=4):   0%|          | 0/16205 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4052 [00:00<?, ? examples/s]

Done


In [ ]:
for i in tdc.main_ddict['train']['Department Name']:
    assert i is not None

for i in tdc.main_ddict['validation']['Department Name']:
    assert i is not None
    
for i in tdc.main_ddict['validation']['Rating']:
    assert i >= 3

## Taking a sample from training data

If you only want to extract a training sample of your data, you can use the `trn_size` argument of the method `process_and_tokenize` (or `do_tokenization`). Since we use [sharding](https://huggingface.co/docs/datasets/process#shard) to extract a sample from a DatasetDict, if `trn_size` is a integer, an approximated size will be returned

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataController(dset,
                         main_text='Review Text',
                         sup_types='classification',
                         label_names='Department Name',
                         filter_dict={'Review Text': lambda x: x is not None,
                                      'Department Name': lambda x: x is not None,
                                     },
                         seed=42
                        )
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True,trn_size=1000)

-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----
----- Do <lambda> on Department Name -----
Done
----- Label Encoding -----
Done
-------------------- Train Test Split --------------------
Validation split based on val_ratio
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 3, which is 0.02% of training set
Filtering leaked data out of training set...


Filter (num_proc=4):   0%|          | 0/18102 [00:00<?, ? examples/s]

Done
-------------------- Shuffling and flattening train set --------------------


Flattening the indices (num_proc=4):   0%|          | 0/18099 [00:00<?, ? examples/s]

Done
-------------------- Tokenization --------------------


Map (num_proc=4):   0%|          | 0/1006 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4526 [00:00<?, ? examples/s]

Done


In [ ]:
tdc.main_ddict

DatasetDict({
    train: Dataset({
        features: ['Review Text', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1006
    })
    validation: Dataset({
        features: ['Review Text', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 4526
    })
})

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataController(dset,
                         main_text='Review Text',
                         sup_types='classification',
                         label_names='Department Name',
                         filter_dict={'Review Text': lambda x: x is not None,
                                      'Department Name': lambda x: x is not None,
                                     },
                         seed=42
                        )
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True,trn_size=0.1) # return 10% of the data

-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----
----- Do <lambda> on Department Name -----
Done
----- Label Encoding -----
Done
-------------------- Train Test Split --------------------
Validation split based on val_ratio
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 3, which is 0.02% of training set
Filtering leaked data out of training set...
Done
-------------------- Shuffling and flattening train set --------------------
Done
-------------------- Tokenization --------------------


Map (num_proc=4):   0%|          | 0/1810 [00:00<?, ? examples/s]

Done


In [ ]:
tdc.main_ddict

DatasetDict({
    train: Dataset({
        features: ['Review Text', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1810
    })
    validation: Dataset({
        features: ['Review Text', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 4526
    })
})

## Metadatas concatenation

If we think metadatas can be helpful, we can concatenate them into the front of your text, so that our text classification model is aware of it.

In this example, Let's add 'Title' as our metadata

In [ ]:
df = pd.read_csv('sample_data/Womens_Clothing_Reviews.csv',encoding='utf-8-sig')
tdc = TextDataController.from_df(df,
                                 main_text='Review Text',
                                 sup_types='classification',
                                 label_names='Department Name',
                                 filter_dict={'Review Text': lambda x: x is not None,
                                              'Department Name': lambda x: x is not None,
                                             },
                                 metadatas='Title',
                                 process_metas=True, # to preprocess the metadata (currently it's just empty space stripping and lowercasing),
                                 seed=42
                                )

- Input Validation Precheck -
Data contains missing values!
-----> List of columns and the number of missing values for each
Title              3810
Review Text         845
Division Name        14
Department Name      14
Class Name           14
dtype: int64
Data contains duplicated values!
-----> Number of duplications: 21 rows


In [ ]:
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)

-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----


Filter (num_proc=4):   0%|          | 0/23486 [00:00<?, ? examples/s]

----- Do <lambda> on Department Name -----


Filter (num_proc=4):   0%|          | 0/22641 [00:00<?, ? examples/s]

Done
----- Metadata Simple Processing & Concatenating to Main Content -----


...

Flattening the indices (num_proc=4):   0%|          | 0/18101 [00:00<?, ? examples/s]

Done
-------------------- Tokenization --------------------


Map (num_proc=4):   0%|          | 0/18101 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4526 [00:00<?, ? examples/s]

Done


In [ ]:
tdc.main_ddict['train']['Review Text'][:5]

['beautiful! . I love this top. it was everything i hoped it would be. it is lined so it is not see through in the chest/back; sleeves are sheer. soft. gorgeous color. love the layers. runs large so definitely size down. i am usually a m and ordered the s. i\'m 5\'8" curvy 32dd',
 'very flattering . This dress fits to a t! true to size. very flattering. fabric is soft and comfortable.',
 "the worst . I don't typically write bad reviews, but this dress is so bad and i want to save someone else from buying it. i read the mostly bad reviews and still purchased anyway (my fault i know). the dress is super stiff ( i know denim can be that way and it is possible it would soften up after a few washes). i'm typically a 6/8 and the size small swallowed me, and the xs was big everywhere except through the bust (i ordered both sizes to try). i wouldn't recommend buying this if you are a size 8 or small",
 "love this jacket! . I was on the lookout for a denim jacket when i found this beauty on lin

In [ ]:
tdc.main_ddict['validation']['Review Text'][:5]

[' . Such a fun jacket! great to wear in the spring or to the office as an alternative to a formal blazer. very comfortable!',
 'simple and elegant . I thought this shirt was really feminine and elegant. only downsides is some of the punched out holes had fabric still attached which you have cut off with scissors- otherwise the shirt will snag. and the second issue of bigger importance are the low armholes. lots of bra showing- not really sure how to get around that so i always wear it with a cardigan. but it would be nice not to have to. \r\nother than that it looks nice and pairs nicely with almost anything.',
 'retro and pretty . This top has a bit of a retro flare but so adorable on. looks really cute with a pair of faded boot cut jeans.',
 'summer/fall wear . I first spotted this on an retailer employee, she paired it with a peasant top & wore it open w/jeans & boots- so darn cute. love how this peice transitions from summer to fall. i\'m 5\'4" so i had to order the small petite w

You can add multiple metadatas. Let's say 'Division Name' is the second metadata.

In [ ]:
df = pd.read_csv('sample_data/Womens_Clothing_Reviews.csv',encoding='utf-8-sig')
tdc = TextDataController.from_df(df,
                                 main_text='Review Text',
                                 sup_types='classification',
                                 label_names='Department Name',
                                 filter_dict={'Review Text': lambda x: x is not None,
                                              'Department Name': lambda x: x is not None,
                                             },
                                 metadatas=['Title','Division Name'],
                                 process_metas=True,
                                 seed=42
                                )
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)

- Input Validation Precheck -
Data contains missing values!
-----> List of columns and the number of missing values for each
Title              3810
Review Text         845
Division Name        14
Department Name      14
Class Name           14
dtype: int64
Data contains duplicated values!
-----> Number of duplications: 21 rows
-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----


Filter (num_proc=4):   0%|          | 0/23486 [00:00<?, ? examples/s]

----- Do <lambda> on Department Name -----


Filter (num_proc=4):   0%|          | 0/22641 [00:00<?, ? examples/s]

Done
----- Metadata Simple Processing & Concatenating to Main Content -----


...

Flattening the indices (num_proc=4):   0%|          | 0/18102 [00:00<?, ? examples/s]

Done
-------------------- Tokenization --------------------


Map (num_proc=4):   0%|          | 0/18102 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4526 [00:00<?, ? examples/s]

Done


In [ ]:
tdc.main_ddict['train']['Review Text'][:5]

["general petite . meh . This tunic is way over priced for the style and quality. it fit comfortably (runs a size larger) but it's not really flattering, it jut kind of hangs there looking ok. it is a little too deep of a v cut for a work top as well. this top does not support the price at all. it felt like something i could find at department store for way less. i will be returning it.",
 'general . awesome buy! . I am so happy i took a chance on this jumpsuit! i am post-baby (six weeks) and although i intend on slimming down more i would say that it is flattering even at my current size. and it will only get better! \r\nthe quality and color are great!',
 'general . warm grey . These are a lovely neutral to slightly warm grey pair of jeans from a great line. i wore my usual size without issues.',
 "general . loved it, but it didn't work for me. . I wanted this top to work so bad. unfortunately the way the bust of the top is designed it isn't flattering if you aren't flat chested. it 

In [ ]:
tdc.main_ddict['validation']['Review Text'][:5] # The metadata for this text is None

['general petite .  . Such a fun jacket! great to wear in the spring or to the office as an alternative to a formal blazer. very comfortable!',
 'general petite . simple and elegant . I thought this shirt was really feminine and elegant. only downsides is some of the punched out holes had fabric still attached which you have cut off with scissors- otherwise the shirt will snag. and the second issue of bigger importance are the low armholes. lots of bra showing- not really sure how to get around that so i always wear it with a cardigan. but it would be nice not to have to. \r\nother than that it looks nice and pairs nicely with almost anything.',
 'general . retro and pretty . This top has a bit of a retro flare but so adorable on. looks really cute with a pair of faded boot cut jeans.',
 'general petite . summer/fall wear . I first spotted this on an retailer employee, she paired it with a peasant top & wore it open w/jeans & boots- so darn cute. love how this peice transitions from su

## Label Encodings

### Single-head prediction

We have briefly gone through the simplest case of label encoding, where we only need to predict 1 single label. We call this **single head classification**

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataController(dset,
                         main_text='Review Text',
                         label_names='Department Name',
                         sup_types='classification',                         
                         filter_dict={'Review Text': lambda x: x is not None,
                                      'Department Name': lambda x: x is not None,
                                     },
                         seed=42
                        )
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)

-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----
----- Do <lambda> on Department Name -----
Done
----- Label Encoding -----
Done
-------------------- Train Test Split --------------------
Validation split based on val_ratio
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 3, which is 0.02% of training set
Filtering leaked data out of training set...
Done
-------------------- Shuffling and flattening train set --------------------
Done
-------------------- Tokenization --------------------


Map (num_proc=4):   0%|          | 0/18099 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4526 [00:00<?, ? examples/s]

Done


All label names will be saved in instance variable `label_lists`

In [ ]:
tdc.label_lists

[['Bottoms', 'Dresses', 'Intimate', 'Jackets', 'Tops', 'Trend']]

... and all labels will be encoded

In [ ]:
tdc.main_ddict['validation']['label'][:5]

[2, 4, 4, 1, 1]

We also keep the original labeling, for references

In [ ]:
tdc.main_ddict['validation']['Department Name'][:5]

['Intimate', 'Tops', 'Tops', 'Dresses', 'Dresses']

You can also do **single-head regression**. Let's say we want to predict **Rating**

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')

In [ ]:
dset

Dataset({
    features: ['Clothing ID', 'Age', 'Title', 'Review Text', 'Rating', 'Recommended IND', 'Positive Feedback Count', 'Division Name', 'Department Name', 'Class Name'],
    num_rows: 23486
})

In [ ]:
tdc = TextDataController(dset,
                         main_text='Review Text',
                         sup_types='regression',
                         label_names='Rating',
                         filter_dict={'Review Text': lambda x: x is not None},
                         seed=42,
                        )

In [ ]:
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)

-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----
Done
----- Label Encoding -----
Done
-------------------- Train Test Split --------------------
Validation split based on val_ratio
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 1, which is 0.01% of training set
Filtering leaked data out of training set...
Done
-------------------- Shuffling and flattening train set --------------------
Done
-------------------- Tokenization --------------------


Map (num_proc=4):   0%|          | 0/18111 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4529 [00:00<?, ? examples/s]

Done


In [ ]:
print(tdc.main_ddict['train']['Rating'][:5])
print(tdc.main_ddict['train']['label'][:5])

[3.0, 1.0, 4.0, 3.0, 5.0]
[3.0, 1.0, 4.0, 3.0, 5.0]


In [ ]:
print(tdc.main_ddict['validation']['Rating'][:5])
print(tdc.main_ddict['validation']['label'][:5])

[5.0, 4.0, 3.0, 5.0, 5.0]
[5.0, 4.0, 3.0, 5.0, 5.0]


### Multi-head prediction

What if we need to predict 2 different labels as once? We call this **multi-head classification/regression**. For example, let's define our dataset so that we need to predict both `Department Name` and `Division Name` (both as classification)

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataController(dset,
                         main_text='Review Text',
                         label_names=['Division Name','Department Name'],
                         sup_types=['classification','classification'],
                         filter_dict={'Review Text': lambda x: x is not None,
                                      'Department Name': lambda x: x is not None,
                                     },
                         seed=42
                        )
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)

-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----
----- Do <lambda> on Department Name -----
Done
----- Label Encoding -----


Map (num_proc=4):   0%|          | 0/22628 [00:00<?, ? examples/s]

Done
-------------------- Train Test Split --------------------
Validation split based on val_ratio
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 3, which is 0.02% of training set
Filtering leaked data out of training set...


Filter (num_proc=4):   0%|          | 0/18102 [00:00<?, ? examples/s]

Done
-------------------- Shuffling and flattening train set --------------------


Flattening the indices (num_proc=4):   0%|          | 0/18099 [00:00<?, ? examples/s]

Done
-------------------- Tokenization --------------------


Map (num_proc=4):   0%|          | 0/18099 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4526 [00:00<?, ? examples/s]

Done


In [ ]:
tdc.label_lists

[['General', 'General Petite', 'Initmates'],
 ['Bottoms', 'Dresses', 'Intimate', 'Jackets', 'Tops', 'Trend']]

We can see that we have two lists, one for label names of `Division Name`, and one for label names of `Department Name`

In [ ]:
tdc.main_ddict

DatasetDict({
    train: Dataset({
        features: ['Review Text', 'Division Name', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 18099
    })
    validation: Dataset({
        features: ['Review Text', 'Division Name', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 4526
    })
})

In [ ]:
print(tdc.main_ddict['validation']['Division Name'][:5])
print(tdc.main_ddict['validation']['Department Name'][:5])
print(tdc.main_ddict['validation']['label'][:5])

['General Petite', 'General Petite', 'General', 'General Petite', 'General Petite']
['Intimate', 'Tops', 'Tops', 'Dresses', 'Dresses']
[[1, 2], [1, 4], [0, 4], [1, 1], [1, 1]]


What if one label is classification, and another label is regression? We will predict `Department Name` (classification) and `Rating` (regression)

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataController(dset,
                         main_text='Review Text',
                         label_names=['Rating','Department Name'],
                         sup_types=['regression','classification'],
                         filter_dict={'Review Text': lambda x: x is not None,
                                      'Department Name': lambda x: x is not None,
                                     },
                         seed=42
                        )
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)

-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----
----- Do <lambda> on Department Name -----
Done
----- Label Encoding -----


Map (num_proc=4):   0%|          | 0/22628 [00:00<?, ? examples/s]

Done
-------------------- Train Test Split --------------------
Validation split based on val_ratio
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 3, which is 0.02% of training set
Filtering leaked data out of training set...


Filter (num_proc=4):   0%|          | 0/18102 [00:00<?, ? examples/s]

Done
-------------------- Shuffling and flattening train set --------------------


Flattening the indices (num_proc=4):   0%|          | 0/18099 [00:00<?, ? examples/s]

Done
-------------------- Tokenization --------------------


Map (num_proc=4):   0%|          | 0/18099 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4526 [00:00<?, ? examples/s]

Done


In [ ]:
print(tdc.main_ddict['validation']['Rating'][:5])
print(tdc.main_ddict['validation']['Department Name'][:5])
print(tdc.main_ddict['validation']['label'][:5])

[5.0, 5.0, 5.0, 5.0, 4.0]
['Intimate', 'Tops', 'Tops', 'Dresses', 'Dresses']
[[5.0, 2.0], [5.0, 4.0], [5.0, 4.0], [5.0, 1.0], [4.0, 1.0]]


Since it's multi-head, you can define multiple classification/regression labels, as many as you want

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataController(dset,
                         main_text='Review Text',
                         label_names=['Division Name','Rating','Department Name'],
                         sup_types=['classification','regression','classification'],
                         filter_dict={'Review Text': lambda x: x is not None,
                                      'Department Name': lambda x: x is not None,
                                      'Division Name': lambda x: x is not None,
                                     },
                         seed=42
                        )
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)

-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----
----- Do <lambda> on Department Name -----
----- Do <lambda> on Division Name -----
Done
----- Label Encoding -----


Map (num_proc=4):   0%|          | 0/22628 [00:00<?, ? examples/s]

Done
-------------------- Train Test Split --------------------
Validation split based on val_ratio
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 3, which is 0.02% of training set
Filtering leaked data out of training set...


Filter (num_proc=4):   0%|          | 0/18102 [00:00<?, ? examples/s]

Done
-------------------- Shuffling and flattening train set --------------------


Flattening the indices (num_proc=4):   0%|          | 0/18099 [00:00<?, ? examples/s]

Done
-------------------- Tokenization --------------------


Map (num_proc=4):   0%|          | 0/18099 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4526 [00:00<?, ? examples/s]

Done


In [ ]:
print(tdc.main_ddict['train']['Division Name'][:5])
print(tdc.main_ddict['train']['Rating'][:5])
print(tdc.main_ddict['train']['Department Name'][:5])
print(tdc.main_ddict['train']['label'][:5])

['General Petite', 'General Petite', 'General', 'General', 'General Petite']
[4.0, 4.0, 5.0, 3.0, 5.0]
['Tops', 'Tops', 'Tops', 'Tops', 'Dresses']
[[1.0, 4.0, 4.0], [1.0, 4.0, 4.0], [0.0, 5.0, 4.0], [0.0, 3.0, 4.0], [1.0, 5.0, 1.0]]


In [ ]:
print(tdc.main_ddict['validation']['Division Name'][:5])
print(tdc.main_ddict['validation']['Rating'][:5])
print(tdc.main_ddict['validation']['Department Name'][:5])
print(tdc.main_ddict['validation']['label'][:5])

['General Petite', 'General Petite', 'General', 'General Petite', 'General Petite']
[5.0, 5.0, 5.0, 5.0, 4.0]
['Intimate', 'Tops', 'Tops', 'Dresses', 'Dresses']
[[1.0, 5.0, 2.0], [1.0, 5.0, 4.0], [0.0, 5.0, 4.0], [1.0, 5.0, 1.0], [1.0, 4.0, 1.0]]


### Multi-label classification

Lastly, let's define a **multi-label classification**, where a text can have 1 or more label. Our data don't have such labeling, so we will make a new one, just for demonstration.

In [ ]:
df = pd.read_csv('sample_data/Womens_Clothing_Reviews.csv',encoding='utf-8-sig')

In [ ]:
df['Department Name'].unique()

array(['Intimate', 'Dresses', 'Bottoms', 'Tops', 'Jackets', 'Trend', nan],
      dtype=object)

In [ ]:
df['Fake Label'] = [np.random.choice(df['Department Name'].unique()[:-1],size=np.random.randint(2,6),replace=False) for _ in range(len(df))]

In [ ]:
df.head()

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name,Fake Label
0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates,"[Bottoms, Intimate]"
1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses,"[Trend, Dresses, Tops, Intimate, Jackets]"
2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses,"[Jackets, Tops, Dresses, Intimate]"
3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants,"[Dresses, Tops, Bottoms, Trend]"
4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses,"[Tops, Jackets, Trend, Dresses]"


You don't have to add any extra argument; the controller will determine whether this is for multilabel classification, based on the format of the label values

In [ ]:
tdc = TextDataController.from_df(df,
                                 main_text='Review Text',
                                 filter_dict={'Review Text': lambda x: x is not None},
                                 label_names='Fake Label',
                                 sup_types='classification',
                                 seed=42,
                                )
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)

- Input Validation Precheck -
Data contains missing values!
-----> List of columns and the number of missing values for each
Title              3810
Review Text         845
Division Name        14
Department Name      14
Class Name           14
dtype: int64
-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----


Filter (num_proc=4):   0%|          | 0/23486 [00:00<?, ? examples/s]

Done
----- Label Encoding -----


Map (num_proc=4):   0%|          | 0/22641 [00:00<?, ? examples/s]

Done
-------------------- Train Test Split --------------------
Validation split based on val_ratio
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 1, which is 0.01% of training set
Filtering leaked data out of training set...


...

Flattening the indices (num_proc=4):   0%|          | 0/18111 [00:00<?, ? examples/s]

Done
-------------------- Tokenization --------------------


Map (num_proc=4):   0%|          | 0/18111 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4529 [00:00<?, ? examples/s]

Done


In [ ]:
tdc.main_ddict

DatasetDict({
    train: Dataset({
        features: ['Review Text', 'Fake Label', 'label', 'input_ids', 'attention_mask'],
        num_rows: 18111
    })
    validation: Dataset({
        features: ['Review Text', 'Fake Label', 'label', 'input_ids', 'attention_mask'],
        num_rows: 4529
    })
})

In [ ]:
tdc.label_lists

[['Bottoms', 'Dresses', 'Intimate', 'Jackets', 'Tops', 'Trend']]

In [ ]:
tdc.main_ddict['validation']['Fake Label'][2]

['Bottoms', 'Tops', 'Jackets', 'Trend']

Since this is **multilabel classification**, the label will be one-hot encoded

In [ ]:
tdc.main_ddict['validation']['label'][2]

[1, 0, 0, 1, 1, 1]

In [ ]:
tdc.main_ddict['validation']['label'][:5]

[[1, 0, 1, 1, 0, 1],
 [1, 1, 0, 1, 0, 1],
 [1, 0, 0, 1, 1, 1],
 [1, 0, 1, 1, 1, 1],
 [1, 0, 1, 1, 0, 1]]

### No label

If you don't have a label to define, leave all label-related arguments blank

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataController(dset,
                         main_text='Review Text',
                         filter_dict={'Review Text': lambda x: x is not None},
                         seed=42
                        )
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)

-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----
Done
-------------------- Train Test Split --------------------
Validation split based on val_ratio
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 1, which is 0.01% of training set
Filtering leaked data out of training set...


Filter (num_proc=4):   0%|          | 0/18112 [00:00<?, ? examples/s]

Done
-------------------- Shuffling and flattening train set --------------------


Flattening the indices (num_proc=4):   0%|          | 0/18111 [00:00<?, ? examples/s]

Done
-------------------- Tokenization --------------------


Map (num_proc=4):   0%|          | 0/18111 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4529 [00:00<?, ? examples/s]

Done


In [ ]:
tdc.main_ddict

DatasetDict({
    train: Dataset({
        features: ['Review Text', 'input_ids', 'attention_mask'],
        num_rows: 18111
    })
    validation: Dataset({
        features: ['Review Text', 'input_ids', 'attention_mask'],
        num_rows: 4529
    })
})

## Label transformation

Sometimes, you want to apply some light transformation to your label(s) before apply label encoding, e.g. there are some typos in your string label (classification), or you want to scale your regression label. `TextDataController` provides a way for you to do so, via `label_tfm_dict` argument. For the following example, I will fix the typo 'Initmates' in `Division Name` label, and log scale the `Rating`

In [ ]:
import math

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataController(dset,
                         main_text='Review Text',
                         label_names=['Division Name','Rating','Department Name'],
                         sup_types=['classification','regression','classification'],
                         filter_dict={'Review Text': lambda x: x is not None,
                                      'Department Name': lambda x: x is not None,
                                      'Division Name': lambda x: x is not None,
                                     },
                         label_tfm_dict={'Division Name': lambda x: x if x!='Initmates' else 'Intimates',
                                         'Rating': lambda x: math.log(x)+1},
                         seed=42,
                         num_proc=1
                        )
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)

-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----
----- Do <lambda> on Department Name -----
----- Do <lambda> on Division Name -----
Done
-------------------- Label Transformation --------------------


Map:   0%|          | 0/22628 [00:00<?, ? examples/s]

Map:   0%|          | 0/22628 [00:00<?, ? examples/s]

Done
----- Label Encoding -----


Map:   0%|          | 0/22628 [00:00<?, ? examples/s]

...

Flattening the indices:   0%|          | 0/18099 [00:00<?, ? examples/s]

Done
-------------------- Tokenization --------------------


Map:   0%|          | 0/18099 [00:00<?, ? examples/s]

Map:   0%|          | 0/4526 [00:00<?, ? examples/s]

Done


Notice that in the label_lists, the label 'Initmates' has been replaced by 'Intimates'

Also, the second empty list corresponds to the label value of `Rating`, which is for regression, thus results in an empty list

In [ ]:
tdc.label_lists

[['General', 'General Petite', 'Intimates'],
 [],
 ['Bottoms', 'Dresses', 'Intimate', 'Jackets', 'Tops', 'Trend']]

In [ ]:
print(tdc.main_ddict['train']['Division Name'][:5])
print(tdc.main_ddict['train']['Rating'][:5])
print(tdc.main_ddict['train']['Department Name'][:5])
print(tdc.main_ddict['train']['label'][:5])

['General Petite', 'General Petite', 'General', 'General', 'General Petite']
[2.386294361119891, 2.386294361119891, 2.6094379124341005, 2.09861228866811, 2.6094379124341005]
['Tops', 'Tops', 'Tops', 'Tops', 'Dresses']
[[1.0, 2.386294361119891, 4.0], [1.0, 2.386294361119891, 4.0], [0.0, 2.6094379124341005, 4.0], [0.0, 2.09861228866811, 4.0], [1.0, 2.6094379124341005, 1.0]]


In [ ]:
print(tdc.main_ddict['validation']['Division Name'][:5])
print(tdc.main_ddict['validation']['Rating'][:5])
print(tdc.main_ddict['validation']['Department Name'][:5])
print(tdc.main_ddict['validation']['label'][:5])

['General Petite', 'General Petite', 'General', 'General Petite', 'General Petite']
[2.6094379124341005, 2.6094379124341005, 2.6094379124341005, 2.6094379124341005, 2.386294361119891]
['Intimate', 'Tops', 'Tops', 'Dresses', 'Dresses']
[[1.0, 2.6094379124341005, 2.0], [1.0, 2.6094379124341005, 4.0], [0.0, 2.6094379124341005, 4.0], [1.0, 2.6094379124341005, 1.0], [1.0, 2.386294361119891, 1.0]]


## Content Transformation

This processing allows you to **alter the text content in your dataset**. You need to define a function that accepts a single string and returns a new, processed string. Note that this transformation will be applied to ALL of your dataset (both train and validation)

Let's say we want to normalize our text, because the text might contain some extra spaces between words, or not follow the "single space after a period" rule

In [ ]:
_tmp = "This is a      sentence,which doesn't follow any rule!No single space is provided after period or punctuation marks.    Maybe there are too many spaces!?!   "

In [ ]:
from underthesea import text_normalize

In [ ]:
text_normalize(_tmp)

"This is a sentence , which doesn't follow any rule ! No single space is provided after period or punctuation marks . Maybe there are too many spaces ! ? !"

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataController(dset,
                         main_text='Review Text',
                         label_names='Department Name',
                         sup_types='classification',
                         filter_dict={'Review Text': lambda x: x is not None,
                                      'Department Name': lambda x: x is not None,
                                     },
                         content_transformations=text_normalize,
                         seed=42
                        )
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)

-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----
----- Do <lambda> on Department Name -----
Done
----- Label Encoding -----
Done
-------------------- Text Transformation --------------------
----- text_normalize -----
Done
-------------------- Train Test Split --------------------
Validation split based on val_ratio
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 3, which is 0.02% of training set
Filtering leaked data out of training set...


Filter (num_proc=4):   0%|          | 0/18102 [00:00<?, ? examples/s]

Done
-------------------- Shuffling and flattening train set --------------------


Flattening the indices (num_proc=4):   0%|          | 0/18099 [00:00<?, ? examples/s]

Done
-------------------- Tokenization --------------------


Map (num_proc=4):   0%|          | 0/18099 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4526 [00:00<?, ? examples/s]

Done


In [ ]:
tdc.main_ddict['train']['Review Text'][0]

"This sweater is beautiful , but is definitely more for looks than warmth . it's very soft , but very thin . i prefer the way it looks open rather than buttoned . i got the moss green color on sale , and i am glad i didn't pay full price for it--it ' s lovely , but certainly not worth $ 88 ."

In [ ]:
tdc.main_ddict['validation']['Review Text'][0]

'Such a fun jacket ! great to wear in the spring or to the office as an alternative to a formal blazer . very comfortable !'

You can chain multiple functions. Let's say after text normalizing, I want to lowercase the text

In [ ]:
str.lower('tHis IS NoT lowerCASE')

'this is not lowercase'

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataController(dset,
                         main_text='Review Text',
                         label_names='Department Name',
                         sup_types='classification',
                         filter_dict={'Review Text': lambda x: x is not None,
                                      'Department Name': lambda x: x is not None,
                                     },
                         content_transformations=[text_normalize,str.lower],
                         seed=42
                        )
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)

-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----
----- Do <lambda> on Department Name -----
Done
----- Label Encoding -----
Done
-------------------- Text Transformation --------------------
----- text_normalize -----
----- lower -----
Done
-------------------- Train Test Split --------------------
Validation split based on val_ratio
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 3, which is 0.02% of training set
Filtering leaked data out of training set...


Filter (num_proc=4):   0%|          | 0/18102 [00:00<?, ? examples/s]

Done
-------------------- Shuffling and flattening train set --------------------


Flattening the indices (num_proc=4):   0%|          | 0/18099 [00:00<?, ? examples/s]

Done
-------------------- Tokenization --------------------


Map (num_proc=4):   0%|          | 0/18099 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4526 [00:00<?, ? examples/s]

Done


In [ ]:
tdc.main_ddict['train']['Review Text'][0]

"this sweater is beautiful , but is definitely more for looks than warmth . it's very soft , but very thin . i prefer the way it looks open rather than buttoned . i got the moss green color on sale , and i am glad i didn't pay full price for it--it ' s lovely , but certainly not worth $ 88 ."

In [ ]:
tdc.main_ddict['validation']['Review Text'][0]

'such a fun jacket ! great to wear in the spring or to the office as an alternative to a formal blazer . very comfortable !'

You can even perform some complex transformations, such as removing text inside parentheses, or replacing some texts via a pattern (which is doable using regular expression). Let's make an example of such transformations, where we remove text inside parentheses, and convert any hashtag into the string 'hashtag'

In [ ]:
import re

In [ ]:
def process_text(s):
    # Remove texts inside parentheses
    s = re.sub(r'\(.*?\)', '', s)
    
    # Convert any hashtag into the string 'hashtag'
    s = re.sub(r'#\w+', 'hashtag', s)
    
    return s.strip()

In [ ]:
process_text("#Promotions There's no way it works (I checked!), however it surprises me #howonearth #mindblowing")

"hashtag There's no way it works , however it surprises me hashtag hashtag"

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataController(dset,
                         main_text='Review Text',
                         label_names='Department Name',
                         sup_types='classification',
                         filter_dict={'Review Text': lambda x: x is not None,
                                      'Department Name': lambda x: x is not None,
                                     },
                         content_transformations=process_text,
                         seed=42
                        )
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)

-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----
----- Do <lambda> on Department Name -----
Done
----- Label Encoding -----
Done
-------------------- Text Transformation --------------------
----- process_text -----
Done
-------------------- Train Test Split --------------------
Validation split based on val_ratio
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 3, which is 0.02% of training set
Filtering leaked data out of training set...


Filter (num_proc=4):   0%|          | 0/18102 [00:00<?, ? examples/s]

Done
-------------------- Shuffling and flattening train set --------------------


Flattening the indices (num_proc=4):   0%|          | 0/18099 [00:00<?, ? examples/s]

Done
-------------------- Tokenization --------------------


Map (num_proc=4):   0%|          | 0/18099 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4526 [00:00<?, ? examples/s]

Done


## Train/Validation Split

There are several ways to perform a train/validation split with `TextDataController`

The first way is when you already have a validation split in your HuggingFace's Dataset. Let's use the Dataset built-in function `train_test_split` to simulate this

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
ddict_with_val = dset.train_test_split(test_size=0.1)
# This will create a 'test' split instead of 'validation', so we will process a bit to have a validation split
ddict_with_val['validation']=ddict_with_val['test']
del ddict_with_val['test']

In [ ]:
ddict_with_val

DatasetDict({
    train: Dataset({
        features: ['Clothing ID', 'Age', 'Title', 'Review Text', 'Rating', 'Recommended IND', 'Positive Feedback Count', 'Division Name', 'Department Name', 'Class Name'],
        num_rows: 21137
    })
    validation: Dataset({
        features: ['Clothing ID', 'Age', 'Title', 'Review Text', 'Rating', 'Recommended IND', 'Positive Feedback Count', 'Division Name', 'Department Name', 'Class Name'],
        num_rows: 2349
    })
})

In [ ]:
tdc = TextDataController(ddict_with_val,
                         main_text='Review Text',
                         label_names='Department Name',
                         sup_types='classification',
                         filter_dict={'Review Text': lambda x: x is not None,
                                      'Department Name': lambda x: x is not None,
                                     },
                         seed=42
                        )
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)

-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----


Filter (num_proc=4):   0%|          | 0/21137 [00:00<?, ? examples/s]

Filter (num_proc=4):   0%|          | 0/2349 [00:00<?, ? examples/s]

----- Do <lambda> on Department Name -----


Filter (num_proc=4):   0%|          | 0/20373 [00:00<?, ? examples/s]

...

Flattening the indices (num_proc=4):   0%|          | 0/20359 [00:00<?, ? examples/s]

Done
-------------------- Tokenization --------------------


Map (num_proc=4):   0%|          | 0/20359 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/2267 [00:00<?, ? examples/s]

Done


In [ ]:
tdc.main_ddict

DatasetDict({
    train: Dataset({
        features: ['Review Text', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 20359
    })
    validation: Dataset({
        features: ['Review Text', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2267
    })
})

A second way is to split randomly based on a ratio (a float between 0 and 1), or based on the number of data in your validation set

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataController(dset,
                         main_text='Review Text',
                         label_names='Department Name',
                         sup_types='classification',
                         filter_dict={'Review Text': lambda x: x is not None,
                                      'Department Name': lambda x: x is not None,
                                     },
                         val_ratio=0.15,
                         seed=42,
                         verbose=False
                        )
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)
print(tdc.main_ddict)

Filter (num_proc=4):   0%|          | 0/19233 [00:00<?, ? examples/s]

Flattening the indices (num_proc=4):   0%|          | 0/19231 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/19231 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3395 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Review Text', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 19231
    })
    validation: Dataset({
        features: ['Review Text', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 3395
    })
})


In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataController(dset,
                         main_text='Review Text',
                         label_names='Department Name',
                         sup_types='classification',
                         filter_dict={'Review Text': lambda x: x is not None,
                                      'Department Name': lambda x: x is not None,
                                     },
                         val_ratio=5000,
                         seed=42,
                         verbose=False
                        )
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)
print(tdc.main_ddict)

Filter (num_proc=4):   0%|          | 0/17628 [00:00<?, ? examples/s]

Flattening the indices (num_proc=4):   0%|          | 0/17624 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/17624 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/5000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Review Text', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 17624
    })
    validation: Dataset({
        features: ['Review Text', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 5000
    })
})


A third way is to do a random stratified split (inspired by [sklearn's](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)). Let's do a stratified split based on our label 'Department Name'

In [ ]:
df = pd.read_csv('sample_data/Womens_Clothing_Reviews.csv',encoding='utf-8-sig')

In [ ]:
df['Department Name'].value_counts(normalize=True)

Department Name
Tops        0.445978
Dresses     0.269214
Bottoms     0.161852
Intimate    0.073918
Jackets     0.043967
Trend       0.005070
Name: proportion, dtype: float64

In [ ]:
tdc = TextDataController.from_df(df,
                                 main_text='Review Text',
                                 label_names='Department Name',
                                 sup_types='classification',
                                 filter_dict={'Review Text': lambda x: x is not None,
                                              'Department Name': lambda x: x is not None,
                                             },
                                 val_ratio=0.2,
                                 stratify_cols='Department Name',
                                 seed=42
                                )
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)

- Input Validation Precheck -
Data contains missing values!
-----> List of columns and the number of missing values for each
Title              3810
Review Text         845
Division Name        14
Department Name      14
Class Name           14
dtype: int64
Data contains duplicated values!
-----> Number of duplications: 21 rows
-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----


Filter (num_proc=4):   0%|          | 0/23486 [00:00<?, ? examples/s]

----- Do <lambda> on Department Name -----


Filter (num_proc=4):   0%|          | 0/22641 [00:00<?, ? examples/s]

Done
----- Label Encoding -----


...

Flattening the indices (num_proc=4):   0%|          | 0/18100 [00:00<?, ? examples/s]

Done
-------------------- Tokenization --------------------


Map (num_proc=4):   0%|          | 0/18100 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4526 [00:00<?, ? examples/s]

Done


In [ ]:
pd.Series(tdc.main_ddict['train']['Department Name']).value_counts(normalize=True)

Tops        0.444033
Dresses     0.271602
Bottoms     0.161878
Intimate    0.072983
Jackets     0.044309
Trend       0.005193
Name: proportion, dtype: float64

In [ ]:
pd.Series(tdc.main_ddict['validation']['Department Name']).value_counts(normalize=True)

Tops        0.444101
Dresses     0.271542
Bottoms     0.161732
Intimate    0.073133
Jackets     0.044189
Trend       0.005303
Name: proportion, dtype: float64

You can also use multiple columns for your stratification

In [ ]:
tdc = TextDataController.from_df(df,
                                 main_text='Review Text',
                                 sup_types='classification',
                                 label_names='Department Name',
                                 filter_dict={'Review Text': lambda x: x is not None,
                                              'Department Name': lambda x: x is not None,
                                             },
                                 val_ratio=0.2,
                                 stratify_cols=['Department Name','Rating'],
                                 seed=42,
                                 verbose=False
                                )
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)

- Input Validation Precheck -
Data contains missing values!
-----> List of columns and the number of missing values for each
Title              3810
Review Text         845
Division Name        14
Department Name      14
Class Name           14
dtype: int64
Data contains duplicated values!
-----> Number of duplications: 21 rows


Filter (num_proc=4):   0%|          | 0/23486 [00:00<?, ? examples/s]

Filter (num_proc=4):   0%|          | 0/22641 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/22628 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/22628 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/22628 [00:00<?, ? examples/s]

Filter (num_proc=4):   0%|          | 0/18102 [00:00<?, ? examples/s]

Flattening the indices (num_proc=4):   0%|          | 0/18100 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/18100 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4526 [00:00<?, ? examples/s]

And finally, you can omit any validation split if you specify `val_ratio` as ```None```

In [ ]:
tdc = TextDataController.from_df(df,
                                 main_text='Review Text',
                                 label_names='Department Name',
                                 sup_types='classification',
                                 filter_dict={'Review Text': lambda x: x is not None,
                                              'Department Name': lambda x: x is not None,
                                             },
                                 val_ratio=None,
                                 seed=42
                                )
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)
tdc.main_ddict

- Input Validation Precheck -
Data contains missing values!
-----> List of columns and the number of missing values for each
Title              3810
Review Text         845
Division Name        14
Department Name      14
Class Name           14
dtype: int64
Data contains duplicated values!
-----> Number of duplications: 21 rows
-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----


Filter (num_proc=4):   0%|          | 0/23486 [00:00<?, ? examples/s]

----- Do <lambda> on Department Name -----


Filter (num_proc=4):   0%|          | 0/22641 [00:00<?, ? examples/s]

Done
----- Label Encoding -----


...

Flattening the indices (num_proc=4):   0%|          | 0/22628 [00:00<?, ? examples/s]

Done
-------------------- Tokenization --------------------


Map (num_proc=4):   0%|          | 0/22628 [00:00<?, ? examples/s]

Done


DatasetDict({
    train: Dataset({
        features: ['Review Text', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 22628
    })
})

## Upsampling

This is useful when you have an imbalanced dataset and you want to perform some upsampling (oversampling) on the minority class. In `TextDataController`, you can perform upsampling on any column of the original dataset, and you can even do upsampling on multiple columns at once

Behind the scene, upsampling contains 2 steps; first, the subset of the data is collected based on the filtering condition, and then this subset is concatenated back into the original data

In [ ]:
df = pd.read_csv('sample_data/Womens_Clothing_Reviews.csv',encoding='utf-8-sig')

In [ ]:
df['Department Name'].sample(frac=0.8).value_counts() 
# fraction 0.8 because we only do upsampling on train data, which is 80% of the total data

Department Name
Tops        8377
Dresses     5054
Bottoms     3044
Intimate    1385
Jackets      823
Trend         94
Name: count, dtype: int64

In [ ]:
df['Department Name'].sample(frac=0.8).value_counts(normalize=True)

Department Name
Tops        0.446391
Dresses     0.269134
Bottoms     0.162397
Intimate    0.072863
Jackets     0.043941
Trend       0.005273
Name: proportion, dtype: float64

Let's say I want to upsampling the 'Trend' by the factor of 2 (x2 the amount of 'Trend' data)

In [ ]:
tdc = TextDataController.from_df(df,
                                 main_text='Review Text',
                                 label_names='Department Name',
                                 sup_types='classification',
                                 filter_dict={'Review Text': lambda x: x is not None,
                                              'Department Name': lambda x: x is not None,
                                             },
                                 val_ratio=0.2,
                                 stratify_cols='Department Name',
                                 upsampling_list=[('Department Name',lambda x: x=='Trend')],
                                 seed=42
                                )
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)

- Input Validation Precheck -
Data contains missing values!
-----> List of columns and the number of missing values for each
Title              3810
Review Text         845
Division Name        14
Department Name      14
Class Name           14
dtype: int64
Data contains duplicated values!
-----> Number of duplications: 21 rows
-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----


Filter (num_proc=4):   0%|          | 0/23486 [00:00<?, ? examples/s]

----- Do <lambda> on Department Name -----


Filter (num_proc=4):   0%|          | 0/22641 [00:00<?, ? examples/s]

Done
----- Label Encoding -----


...

Flattening the indices (num_proc=4):   0%|          | 0/18194 [00:00<?, ? examples/s]

Done
-------------------- Tokenization --------------------


Map (num_proc=4):   0%|          | 0/18194 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4526 [00:00<?, ? examples/s]

Done


In [ ]:
pd.Series(tdc.main_ddict['train']['Department Name']).value_counts()

Tops        8037
Dresses     4916
Bottoms     2930
Intimate    1321
Jackets      802
Trend        188
Name: count, dtype: int64

In [ ]:
pd.Series(tdc.main_ddict['train']['Department Name']).value_counts(normalize=True)

Tops        0.441739
Dresses     0.270199
Bottoms     0.161042
Intimate    0.072606
Jackets     0.044080
Trend       0.010333
Name: proportion, dtype: float64

The percenntage of 'Trend' data in the train set has approximately doubled (note that we filter some NaN text value so the result is not exactly doubled)

In [ ]:
pd.Series(tdc.main_ddict['validation']['Department Name']).value_counts(normalize=True)

Tops        0.444101
Dresses     0.271542
Bottoms     0.161732
Intimate    0.073133
Jackets     0.044189
Trend       0.005303
Name: proportion, dtype: float64

Since augmentation is applied only to the train set, the distribution of label in the validation set remains the same

Similarly, you can triple the amount of 'Trend' by repeating the procedure twice. In the following examples, I will triple the 'Trend' and double the 'Jackets'

In [ ]:
tdc = TextDataController.from_df(df,
                                 main_text='Review Text',
                                 label_names='Department Name',
                                 sup_types='classification',
                                 filter_dict={'Review Text': lambda x: x is not None,
                                              'Department Name': lambda x: x is not None,
                                             },
                                 val_ratio=0.2,
                                 stratify_cols='Department Name',
                                 upsampling_list=[('Department Name',lambda x: x=='Trend'),
                                                  ('Department Name',lambda x: x=='Trend'),
                                                  ('Department Name',lambda x: x=='Jackets')
                                                 ],
                                 # This can be simplified as
#                                  upsampling_list=[('Department Name',lambda x: x=='Trend' or x=='Jackets'),
#                                                   ('Department Name',lambda x: x=='Trend')],
                                 seed=42
                                )
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)

- Input Validation Precheck -
Data contains missing values!
-----> List of columns and the number of missing values for each
Title              3810
Review Text         845
Division Name        14
Department Name      14
Class Name           14
dtype: int64
Data contains duplicated values!
-----> Number of duplications: 21 rows
-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----


Filter (num_proc=4):   0%|          | 0/23486 [00:00<?, ? examples/s]

----- Do <lambda> on Department Name -----


Filter (num_proc=4):   0%|          | 0/22641 [00:00<?, ? examples/s]

Done
----- Label Encoding -----


...

Flattening the indices (num_proc=4):   0%|          | 0/19090 [00:00<?, ? examples/s]

Done
-------------------- Tokenization --------------------


Map (num_proc=4):   0%|          | 0/19090 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4526 [00:00<?, ? examples/s]

Done


In [ ]:
pd.Series(tdc.main_ddict['train']['Department Name']).value_counts()

Tops        8037
Dresses     4916
Bottoms     2930
Jackets     1604
Intimate    1321
Trend        282
Name: count, dtype: int64

**A word of warning**: **Upsampling is a slow procedure**, as it requires multiple dataset concatenation.

## Content Augmentation

Similarly to Content Transformation, Content Augmentation allows to alter the text content in your dataset. You also need to provide a function accepting a single string, and return a new, processed string. Unlike Content Transformation which is applied to ALL data, the Content Augmentation only applies to your TRAINING data

One of the popular library for data augmentation is [nlpaug](https://github.com/makcedward/nlpaug). We will demonstrate how to integrate its augmentation functions into our `TextDataController`

In [ ]:
import nlpaug.augmenter.char as nac

In [ ]:
_tmp = "I like my clothes loose fitting but even for me this ran large, i am 5'7 134b and medium fit in the shoulders but was too big overall"

In [ ]:
def nlp_aug(x,aug=None):
    results = aug.augment(x)
    if not isinstance(x,list): return results[0]
    return results

Augmentation by replacing character with nearby one on the keyboard


In [ ]:
aug = nac.KeyboardAug(aug_char_max=3,aug_char_p=0.1,aug_word_p=0.07)
nearby_aug_func = partial(nlp_aug,aug=aug)

In [ ]:
nearby_aug_func(_tmp)

"I liJe my clothSs loose fitting but even for me this ran large, i am 5 ' 7 134b and medium fit in the shou>ders but was too big overall"

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataController(dset,
                         main_text='Review Text',
                         label_names='Department Name',
                         sup_types='classification',
                         filter_dict={'Review Text': lambda x: x is not None,
                                      'Department Name': lambda x: x is not None,
                                     },
                         content_augmentations=nearby_aug_func,
                         seed=42,
                         verbose=True
                        )
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)

-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----
----- Do <lambda> on Department Name -----
Done
----- Label Encoding -----
Done
-------------------- Train Test Split --------------------
Validation split based on val_ratio
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 3, which is 0.02% of training set
Filtering leaked data out of training set...
Done
-------------------- Text Augmentation --------------------
----- nlp_aug -----


Map (num_proc=4):   0%|          | 0/18099 [00:00<?, ? examples/s]

Done
-------------------- Shuffling and flattening train set --------------------


Flattening the indices (num_proc=4):   0%|          | 0/18099 [00:00<?, ? examples/s]

Done
-------------------- Tokenization --------------------


Map (num_proc=4):   0%|          | 0/18099 [00:00<?, ? examples/s]

Done


In [ ]:
tdc.main_ddict['train']['Review Text'][:5]

["This sweater is beautiful, but is definitely more for looks than waGmth. it ' s very soft, but very thij. i prefer the way it looks open rather than buttoned. i got the moss green color on sale, and i am glad i didn ' t pay full p%ice for it - - it ' s loFely, but certainly not wIrth $ 88.",
 "I ' m a curvy peGson, so my review might not be suited to everyone. my standard size in retailer tops is xl, and it is the same for this blouse. - overall: overall gorgeous, well made blouse but i wish there was lesA fabric onvolved and the burnt out design djdn ' t make a horizontal stripe across the back and biceps. this blouse just might not wotk out as well if you are a full figured person. - prps: gorgeous blouse high qualigy ^nique - cons: i wish the burnt out design didn ' t make a hor",
 'This blouse is wonderfu.. i just got and wore the wine colored blouse today. i received so mwny compliments. i ,ove it and with the sale price it is so worth it.',
 'When i saw thls, i ordered immediat

Again, since this is Content Augmentation, the validation set is unmodified.

In [ ]:
tdc.main_ddict['validation']['Review Text'][:5]

['Such a fun jacket! great to wear in the spring or to the office as an alternative to a formal blazer. very comfortable!',
 'I thought this shirt was really feminine and elegant. only downsides is some of the punched out holes had fabric still attached which you have cut off with scissors- otherwise the shirt will snag. and the second issue of bigger importance are the low armholes. lots of bra showing- not really sure how to get around that so i always wear it with a cardigan. but it would be nice not to have to. \r\nother than that it looks nice and pairs nicely with almost anything.',
 'This top has a bit of a retro flare but so adorable on. looks really cute with a pair of faded boot cut jeans.',
 'I first spotted this on an retailer employee, she paired it with a peasant top & wore it open w/jeans & boots- so darn cute. love how this peice transitions from summer to fall. i\'m 5\'4" so i had to order the small petite which is perfect. note that this dress is very long! it\'s just

You can even apply Content Augmentation stochastically, by adding a random condition in your augmentation function

In [ ]:
# def nlp_aug_stochastic(x,aug=None,p=0.5):
#     results = aug.augment(x)
#     if not isinstance(x,list): return results[0] if random.random()<p else x
#     return [a if random.random()<p else b for a,b in zip(results,x)]

In [ ]:
def nlp_aug_stochastic(x,aug=None,p=0.5):
    if not isinstance(x,list): 
        if random.random()<p: return aug.augment(x)[0]
        return x
    news=[]
    originals=[]
    for _x in x:
        if random.random()<p: news.append(_x)
        else: originals.append(_x)
    # only perform augmentation when needed
    if len(news): news = aug.augment(news)
    return news+originals

In [ ]:
aug = nac.KeyboardAug(aug_char_max=3,aug_char_p=0.1,aug_word_p=0.07)
nearby_aug_func = partial(nlp_aug_stochastic,aug=aug,p=0.3) # nearby_augmentation only applies 30% of the time, with p=0.3

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataController(dset,
                         main_text='Review Text',
                         label_names='Department Name',
                         sup_types='classification',
                         filter_dict={'Review Text': lambda x: x is not None,
                                      'Department Name': lambda x: x is not None,
                                     },
                         content_augmentations=nearby_aug_func,
                         seed=42
                        )
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)

-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----
----- Do <lambda> on Department Name -----
Done
----- Label Encoding -----
Done
-------------------- Train Test Split --------------------
Validation split based on val_ratio
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 3, which is 0.02% of training set
Filtering leaked data out of training set...
Done
-------------------- Text Augmentation --------------------
----- nlp_aug_stochastic -----


Map (num_proc=4):   0%|          | 0/18099 [00:00<?, ? examples/s]

Done
-------------------- Shuffling and flattening train set --------------------


Flattening the indices (num_proc=4):   0%|          | 0/18099 [00:00<?, ? examples/s]

Done
-------------------- Tokenization --------------------


Map (num_proc=4):   0%|          | 0/18099 [00:00<?, ? examples/s]

Done


In [ ]:
tdc.main_ddict['train']['Review Text'][:10]

["This sweater is beautiful, but is definitely more for looks than warmth. it's very soft, but very thin. i prefer the way it looks open rather than buttoned. i got the moss green color on sale, and i am glad i didn't pay full price for it--it's lovely, but certainly not worth $88.",
 "I ' m a curvy person, so my review might not be suited to everyone. my standard size in retailer tops is xl, and it is the same for this blouse. - overall: overall gorgeous, well made blouse but i wish there was ?ess fabric involved and the burn% out desitn didn ' t make a horizontal stripe acfoss the back and biceps. this blouse Hust might not woek out as well if you are a full f8gured person. - pris: gorgeous blouse high quality unique - cons: i wish the burnt out design didn ' t make a hor",
 'This blouse is wonde#ful. i just got and wore the wine colored blouse today. i received so <any complimWntx. i love it and with the sale price it is so worth it.',
 'When i saw thiA, i ordered immediately thinki

One of the advanced augmentation is "Contexttual Word Embeddings Augmenter" (code example: https://github.com/makcedward/nlpaug/blob/master/example/textual_augmenter.ipynb), where you can insert/substitute words using LLM such as BERT, RoBERTA ...

In [ ]:
import nlpaug.augmenter.word as naw

In [ ]:
aug = naw.ContextualWordEmbsAug(model_path='roberta-base', 
                                device='cuda:0', # if you don't have gpu, change to 'cpu'
                                action="substitute",
                                top_k=10,
                               aug_p=0.07)

In [ ]:
contextual_aug_func = partial(nlp_aug,aug=aug)

In [ ]:
_tmp = "I like my clothes loose fitting but even for me this ran large, i am 5'7 134b and medium fit in the shoulders but was too big overall"

In [ ]:
contextual_aug_func(_tmp)

"I kept my clothes slim fitting but even for me this ran large, i am 5'7 134b and medium fit in upper shoulders but was too big overall"

In [ ]:
contextual_aug_func([_tmp for i in range(7)])

["I like my clothes big enough but even for me this ran large, i am 5'7 134b and medium fit in the shoulders but felt too big overall",
 "I like my clothes loose fitting but even for me this ran large, i am 5'7 134b and I fit in the back but still too big overall",
 "I like my big loose fitting but even for me this ran large, i stand 5'7 134b and medium light in the shoulders but was too big overall",
 "I liked its own loose fitting but even for me this ran large, i am 5'7 134b and medium fit in the shoulders but was too big overall",
 "I like my clothes loose fitting but had given me this ran large, i am 5'7 134b is medium fit in the shoulders but was too big overall",
 "I made my clothes loose fitting but even for me this ran large, i am 5'7 134b and barely fit over the shoulders but was too big overall",
 "I like my clothes loose fitting but honestly for me this ran large, i am 5'7 134b and medium fit in all shoulders it was too big overall"]

For this type of augmentation, it's wise to use GPU to minimize processing speed. You also don't want all of your text to be augmented, so let's reuse the stochastic augmentation.

In [ ]:
contextual_aug_func = partial(nlp_aug_stochastic,aug=aug,p=0.3)

In [ ]:
# add these 2 instance variables to your gpu augmentation
contextual_aug_func.run_on_gpu=True
contextual_aug_func.batch_size=32

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataController(dset,
                         main_text='Review Text',
                         sup_types='classification',
                         label_names='Department Name',
                         filter_dict={'Review Text': lambda x: x is not None,
                                      'Department Name': lambda x: x is not None,
                                     },
                         content_augmentations=contextual_aug_func,
                         seed=42
                        )
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)

-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----
----- Do <lambda> on Department Name -----
Done
----- Label Encoding -----
Done
-------------------- Train Test Split --------------------
Validation split based on val_ratio
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 3, which is 0.02% of training set
Filtering leaked data out of training set...
Done
-------------------- Text Augmentation --------------------
----- nlp_aug_stochastic -----


Map:   0%|          | 0/18099 [00:00<?, ? examples/s]

Done
-------------------- Shuffling and flattening train set --------------------


Flattening the indices (num_proc=4):   0%|          | 0/18099 [00:00<?, ? examples/s]

Done
-------------------- Tokenization --------------------


Map (num_proc=4):   0%|          | 0/18099 [00:00<?, ? examples/s]

Done


In [ ]:
tdc.main_ddict['train']['Review Text'][:10]

['This dress makes me so sad...the textured stretchy fabric, color, length, and overall swingy fit are spot on. as another reviewer noted though, the armholes and neck totally ruin the dress. the neck is tiny, which i could have gotten over once it was on, but the arm holes were just awful - too big all around (too tall of a cut plus too wide of a cut). basically, you could see my bra plus from the front there was unflattering exposure near the armholes. it could have been so good, but alas, but t',
 "This top is very flattering, the fabric flows and moves. it fits perfectly (slim cut), but hides tummy bulges and other imperfections. and it's slimming too. can be dressed up or down, goes with everything. i ended up buying all three colors, and if there were more, i would buy more!",
 'This blouse is wonderful. i just got and wore the wine colored blouse today. i received so many compliments. i love it and with the sale price it is so worth it.',
 'This top is very versatile. i wore it 

And finally, similarly to `Content Transformation`, you can link multiple augmentation functions together by providing a list of those functions in `content_augmentations`

## Save and Load TextDataController

In [ ]:
show_doc(TextDataController.save_as_pickles)

---

[source](https://github.com/anhquan0412/that-nlp-library/blob/main/that_nlp_library/text_main.py#L218){target="_blank" style="float:right; font-size:smaller"}

### TextDataController.save_as_pickles

>      TextDataController.save_as_pickles (fname, parent='pickle_files',
>                                          drop_attributes=False)

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| fname |  |  | Name of the pickle file |
| parent | str | pickle_files | Parent folder |
| drop_attributes | bool | False | Whether to drop large-size attributes |

In [ ]:
show_doc(TextDataController.from_pickle)

---

[source](https://github.com/anhquan0412/that-nlp-library/blob/main/that_nlp_library/text_main.py#L187){target="_blank" style="float:right; font-size:smaller"}

### TextDataController.from_pickle

>      TextDataController.from_pickle (fname, parent='pickle_files')

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| fname |  |  | Name of the pickle file |
| parent | str | pickle_files | Parent folder |

TextDataController object can be saved and loaded with ease. This is especially useful after text processing and/or tokenization have been done

In [ ]:
from datasets import disable_caching

In [ ]:
disable_caching() # disable huggingface caching to see data size

In [ ]:
from underthesea import text_normalize
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw

In [ ]:
def nlp_aug_stochastic(x,aug=None,p=0.5):
    if not isinstance(x,list): 
        if random.random()<p: return aug.augment(x)[0]
        return x
    news=[]
    originals=[]
    for _x in x:
        if random.random()<p: news.append(_x)
        else: originals.append(_x)
    # only perform augmentation when needed
    if len(news): news = aug.augment(news)
    return news+originals

In [ ]:
aug2 = naw.ContextualWordEmbsAug(model_path='roberta-base', 
                                device='cuda:0', # if you don't have gpu, change to 'cpu'
                                action="substitute",
                                top_k=10,
                               aug_p=0.07)

contextual_aug_func = partial(nlp_aug_stochastic,aug=aug2,p=0.1)
# add these 2 instance variables to your gpu augmentation
contextual_aug_func.run_on_gpu=True
contextual_aug_func.batch_size=32

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataController(dset,
                         main_text='Review Text',
                         label_names='Department Name',
                         sup_types='classification',
                         filter_dict={'Review Text': lambda x: x is not None,
                                      'Department Name': lambda x: x is not None,
                                     },
                         metadatas=['Title','Division Name'],
                         content_transformations = [text_normalize,str.lower],
                         content_augmentations = contextual_aug_func, 
                         process_metas=True,
                         seed=42
                        )
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)

-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----


Filter (num_proc=4):   0%|          | 0/23486 [00:00<?, ? examples/s]

----- Do <lambda> on Department Name -----


Filter (num_proc=4):   0%|          | 0/22641 [00:00<?, ? examples/s]

Done
----- Metadata Simple Processing & Concatenating to Main Content -----


...

Flattening the indices (num_proc=4):   0%|          | 0/18102 [00:00<?, ? examples/s]

Done
-------------------- Tokenization --------------------


Map (num_proc=4):   0%|          | 0/18102 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4526 [00:00<?, ? examples/s]

Done


In [ ]:
tdc.main_ddict

DatasetDict({
    train: Dataset({
        features: ['Title', 'Review Text', 'Division Name', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 18102
    })
    validation: Dataset({
        features: ['Title', 'Review Text', 'Division Name', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 4526
    })
})

In [ ]:
tdc.save_as_pickles('my_tdc')

Let's check the file size

In [ ]:
file_stats = os.stat(Path('pickle_files/my_tdc.pkl'))
print(f'File Size in MegaBytes is {round(file_stats.st_size / (1024 * 1024), 3)}')

File Size in MegaBytes is 479.387


Load back our object

In [ ]:
tdc2 = TextDataController.from_pickle('my_tdc')

You can still access all its attributes, data, preprocessings, transformation/augmentation ...

In [ ]:
tdc2.main_ddict

DatasetDict({
    train: Dataset({
        features: ['Title', 'Review Text', 'Division Name', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 18102
    })
    validation: Dataset({
        features: ['Title', 'Review Text', 'Division Name', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 4526
    })
})

In [ ]:
for i,v in enumerate(tdc2.main_ddict['train']):
    if i==3:break
    print(f"Text: {v['Review Text']}\nLabel: {v['Department Name']} => {v['label']}")
    print('-'*10)

Text: general petite . meh . this tunic is way over priced for the style and quality . it fit comfortably ( runs a size larger ) but it's not really flattering , it jut kind of hangs there looking ok . it is a little too deep of a v cut for a work top as well . this top does not support the price at all . it felt like something i could find at department store for way less . i will be returning it .
Label: Tops => 4
----------
Text: general petite . love byron lars . this dress , like all byron lars dresses is a work of art . it has true quality of workmanship . and fits like a glove . i always get compliments when i wear any of his dresses and i have 5 ! . this one is somewhere between casual and dressy . perfect for a dinner out on a saturday night ! . order a petite if you are under 5 5 "
Label: Bottoms => 0
----------
Text: general petite . snap neck pullover . i love this top . i ordered it in a large thinking it would be a tight rib but it is not so i reordered it in a small . i 

In [ ]:
tdc2.label_lists

[['Bottoms', 'Dresses', 'Intimate', 'Jackets', 'Tops', 'Trend']]

In [ ]:
tdc2.filter_dict,tdc2.content_tfms,tdc2.aug_tfms

({'Review Text': <function __main__.<lambda>(x)>,
  'Department Name': <function __main__.<lambda>(x)>},
 [<function underthesea.pipeline.text_normalize.text_normalize(text, tokenizer='underthesea')>,
  <method 'lower' of 'str' objects>],
 [functools.partial(<function nlp_aug_stochastic>, aug=<nlpaug.augmenter.word.context_word_embs.ContextualWordEmbsAug object>, p=0.1)])

If you don't want to store the HuggingFace DatasetDict in your `TextDataController`, or the augmentation functions (typically when you already have a trained model, and you only use `TextDataController` to preprocess the test set), you can remove it in the `save_as_pickles` step

In [ ]:
tdc.save_as_pickles('my_lightweight_tdc',drop_attributes=True)

Let's check the file size

In [ ]:
file_stats = os.stat(Path('pickle_files/my_lightweight_tdc.pkl'))
print(f'File Size in MegaBytes is {round(file_stats.st_size / (1024 * 1024), 3)}')

File Size in MegaBytes is 2.279


Load it back

In [ ]:
tdc3 = TextDataController.from_pickle('my_lightweight_tdc')

We will use this object to demonstrate the Test Set Construction in the next section

### Construct a Test Dataset

In [ ]:
show_doc(TextDataController.prepare_test_dataset)

---

[source](https://github.com/anhquan0412/that-nlp-library/blob/main/that_nlp_library/text_main.py#L620){target="_blank" style="float:right; font-size:smaller"}

### TextDataController.prepare_test_dataset

>      TextDataController.prepare_test_dataset (test_dset, do_filtering=False)

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| test_dset |  |  | The HuggingFace Dataset as Test set |
| do_filtering | bool | False | whether to perform data filtering on this test set |

In [ ]:
show_doc(TextDataController.prepare_test_dataset_from_csv)

---

[source](https://github.com/anhquan0412/that-nlp-library/blob/main/that_nlp_library/text_main.py#L587){target="_blank" style="float:right; font-size:smaller"}

### TextDataController.prepare_test_dataset_from_csv

>      TextDataController.prepare_test_dataset_from_csv (file_path,
>                                                        do_filtering=False)

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| file_path |  |  | path to csv file |
| do_filtering | bool | False | whether to perform data filtering on this test set |

In [ ]:
show_doc(TextDataController.prepare_test_dataset_from_df)

---

[source](https://github.com/anhquan0412/that-nlp-library/blob/main/that_nlp_library/text_main.py#L597){target="_blank" style="float:right; font-size:smaller"}

### TextDataController.prepare_test_dataset_from_df

>      TextDataController.prepare_test_dataset_from_df (df, validate=True,
>                                                       do_filtering=False)

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df |  |  | Pandas Dataframe |
| validate | bool | True | whether to perform input data validation |
| do_filtering | bool | False | whether to perform data filtering on this test set |

In [ ]:
show_doc(TextDataController.prepare_test_dataset_from_raws)

---

[source](https://github.com/anhquan0412/that-nlp-library/blob/main/that_nlp_library/text_main.py#L607){target="_blank" style="float:right; font-size:smaller"}

### TextDataController.prepare_test_dataset_from_raws

>      TextDataController.prepare_test_dataset_from_raws (content)

|    | **Details** |
| -- | ----------- |
| content | Either a single sentence, list of sentence or a dictionary with keys are metadata columns and values are list |

Let's say you have done your preprocessing and tokenization in your training set, and have a nicely trained model, ready to do inference on new data. Here is how you can use `TextDataController` to apply all the necessary preprocessings to your new data

We will reuse the lightweight tdc object we created in the previous section (since we don't really need all the training data just to construct new data). Also, we will take a small sample of our training data and pretend it is our test data

In [ ]:
tdc = TextDataController.from_pickle('my_lightweight_tdc')

Let's predict a few raw texts

If we only provide a raw text as follows

```python
tdc.prepare_test_dataset_from_raws('This shirt is so comfortable I love it!')
```
You will counter this error:
```
ValueError: There is/are metadatas in the preprocessing step. Please include a dictionary including these keys for 
metadatas: ['Title', 'Division Name'], and texture content: Review Text
```

Since our preprocessing includes some metadatas, you have to provide a dictionary as follows:

In [ ]:
results = tdc.prepare_test_dataset_from_raws({'Review Text': 'This shirt is so comfortable I love it!',
                                    'Title': 'Great shirt',
                                    'Division Name': 'general'
                                   })

-------------------- Start Test Set Transformation --------------------
----- Metadata Simple Processing & Concatenating to Main Content -----


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Done
-------------------- Text Transformation --------------------
----- text_normalize -----


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

----- lower -----


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Done
-------------------- Tokenization --------------------


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Done


In [ ]:
print(results[0])

{'Review Text': 'general . great shirt . this shirt is so comfortable i love it !', 'Title': 'great shirt', 'Division Name': 'general', 'input_ids': [0, 15841, 479, 372, 6399, 479, 42, 6399, 16, 98, 3473, 939, 657, 24, 27785, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


Let's make prediction from a pandas Dataframe

In [ ]:
df_test = pd.read_csv('sample_data/Womens_Clothing_Reviews.csv',encoding='utf-8-sig').sample(frac=0.2,random_state=1)
# drop NaN values in the label column
df_test = df_test[~df_test['Department Name'].isna()].reset_index(drop=True)
df_test.shape

(4692, 10)

There are few things to pay attention to when constructing your new test set using `TextDataController`:
- Only a few processings will be applied to your test set: **Metadatas concatenation, Filtering (can be omited), Content Transformation, and Tokenization**. Therefore, all columns required to perform these processings must exist in your test dataset
- You can exclude the label column (e.g. `Department Name` in this example), since it's a test set

To view all required columns, access the attribute `cols_to_keep` (you can omit the last column, which is the name of the label column)

In [ ]:
tdc.cols_to_keep

['Review Text', 'Title', 'Division Name', 'Department Name']

This test dataset might have some NaN values in the text field (`Review Text`), thus we will turn on the filtering option to get rid of these NaNs, as this is what we did in the training set. If your test dataset don't need any filtering, turn off this option

In [ ]:
test_dset = tdc.prepare_test_dataset_from_df(df_test,validate=True,do_filtering=True)

- Input Validation Precheck -
Data contains missing values!
-----> List of columns and the number of missing values for each
Title          758
Review Text    164
dtype: int64
Data contains duplicated values!
-----> Number of duplications: 2 rows
-------------------- Start Test Set Transformation --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----


Filter (num_proc=4):   0%|          | 0/4692 [00:00<?, ? examples/s]

----- Do <lambda> on Department Name -----


Filter (num_proc=4):   0%|          | 0/4528 [00:00<?, ? examples/s]

Done
----- Metadata Simple Processing & Concatenating to Main Content -----


...

----- lower -----


Map (num_proc=4):   0%|          | 0/4528 [00:00<?, ? examples/s]

Done
-------------------- Tokenization --------------------


Map (num_proc=4):   0%|          | 0/4528 [00:00<?, ? examples/s]

Done


In [ ]:
test_dset

Dataset({
    features: ['Title', 'Review Text', 'Division Name', 'Department Name', 'input_ids', 'attention_mask'],
    num_rows: 4528
})

In [ ]:
for i in range(3):
    print(f"Text: {test_dset['Review Text'][i]}")
    print(f"Input_ids: {test_dset['input_ids'][i]}")
    print('-'*10)

Text: general . perfect for work and play . this shirt works for both going out and going to work , and i can wear it with everything . fits perfect , tucked and untucked , tied and untied . i love it .
Input_ids: [0, 15841, 479, 1969, 13, 173, 8, 310, 479, 42, 6399, 1364, 13, 258, 164, 66, 8, 164, 7, 173, 2156, 8, 939, 64, 3568, 24, 19, 960, 479, 10698, 1969, 2156, 21222, 8, 7587, 23289, 2156, 3016, 8, 7587, 2550, 479, 939, 657, 24, 479, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
----------
Text: general petite . . i don't know why i had the opposite problem most reviewers had with these ..... i tried on the regular length in the store and found that they were just a bit too short with heels . ( i'm 5 ' 5 ) . i had them ordere

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()